# echuu v3 - AI VTuber 直播引擎

> **V3 改进**: 两阶段沉浸生成 + 随机 Few-shot + 情绪断点 + 认知特征标注

## 核心架构

```
标注数据 → Pattern Analyzer
               ↓
用户输入 (角色设定 + 话题)
               ↓
┌──────────────────────────────────────────────────────┐
│  Phase 1: 预生成完整剧本 (100s)                       │
│  • 10-15句完整台词，有起承转合                        │
│  • 每句标注 key_info（关键信息点）                   │
│  • 设置 interruption_cost                            │
└──────────────────────────────────────────────────────┘
               ↓
┌──────────────────────────────────────────────────────┐
│  Phase 2: 实时表演 + 记忆系统                         │
│                                                       │
│  🧠 记忆系统                                          │
│  ├─ 剧本进度（说到第几句）                            │
│  ├─ 已提到信息（避免重复）                            │
│  ├─ 弹幕记忆（已回应/待回答）                         │
│  └─ 承诺追踪（说了"等会告诉你"要兑现）                │
│                                                       │
│  ⚡ 统一弹幕处理                                      │
│  priority = base + relevance_bonus + sc_bonus        │
│  • 相关性是最重要因素                                 │
│  • 高相关问题 > 低相关SC                              │
│                                                       │
│  🔍 答案查找机制                                      │
│  在 key_info 中匹配问题 → 决定叙事策略                │
│  • 答案还没说：吊胃口 + 记录承诺                      │
│  • 答案在1-3句后：可跳跃（高优先级）                  │
│  • 答案已说过：提醒观众                               │
└──────────────────────────────────────────────────────┘
               ↓
     输出: 台词 + 🔊 语音 + 记忆状态可视化
```

## 核心创新

1. **完整剧本预生成** - 100s完整故事，避免车轱辘话
2. **key_info 标注** - 每句话的关键信息，用于匹配弹幕问题
3. **记忆系统** - 追踪已说内容、弹幕、承诺
4. **统一弹幕处理** - 相关性为王的优先级系统
5. **承诺追踪** - 说了"等会告诉你"必须兑现
6. **答案查找** - 在剧本中定位问题答案，决定叙事策略

## Part 1: 环境准备

In [62]:
# 安装依赖
!pip install anthropic dashscope python-dotenv -q
!pip install requests pydub numpy

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [63]:
import json
import os
import sys
import re
import random
import threading
import base64
from pathlib import Path
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Tuple, Callable
from enum import Enum
from collections import defaultdict, Counter
from datetime import datetime
import io

from IPython.display import display, HTML, Markdown, Audio

# 加载环境变量
from dotenv import load_dotenv

# 自动查找项目根目录
PROJECT_ROOT = Path.cwd()
while PROJECT_ROOT.name != 'echuu-agent' and PROJECT_ROOT.parent != PROJECT_ROOT:
    PROJECT_ROOT = PROJECT_ROOT.parent

load_dotenv(PROJECT_ROOT / '.env')

print(f"📁 项目根目录: {PROJECT_ROOT}")
print(f"🔑 ANTHROPIC_API_KEY: {'✅ 已配置' if os.getenv('ANTHROPIC_API_KEY') else '❌ 未配置'}")
print(f"🔑 DASHSCOPE_API_KEY: {'✅ 已配置' if os.getenv('DASHSCOPE_API_KEY') else '❌ 未配置'}")
print(f"\n✅ 导入完成（已添加 re, random 等V2所需模块）")

# 剧本管理工具
try:
    from script_manager_patch import save_script_to_file, show_script_progress
    scripts_dir = PROJECT_ROOT / "output" / "scripts"
    scripts_dir.mkdir(parents=True, exist_ok=True)
    print("✅ 剧本管理工具已加载")
except ImportError:
    print("⚠️ script_manager_patch 未找到，部分功能不可用")
    save_script_to_file = None
    show_script_progress = None


📁 项目根目录: d:\vtuberclip\echuu-agent
🔑 ANTHROPIC_API_KEY: ✅ 已配置
🔑 DASHSCOPE_API_KEY: ✅ 已配置

✅ 导入完成（已添加 re, random 等V2所需模块）
✅ 剧本管理工具已加载


## Part 2: 数据加载与模式分析

包含：
- PatternAnalyzer（从真实数据学习口语模式）
- 加载标注数据
- 查看示例数据

In [64]:
class PatternAnalyzer:
    """从标注数据中提取模式"""
    
    def __init__(self, annotated_clips: List[Dict]):
        self.clips = annotated_clips
        self.all_segments = []
        for clip in annotated_clips:
            self.all_segments.extend(clip.get("segments", []))
    
    def _normalize_field(self, value, default="self"):
        """处理字段值，如果是列表则取第一个元素"""
        if isinstance(value, list):
            return value[0] if value else default
        return value if value else default
    
    def compute_attention_transitions(self) -> Dict[str, Dict[str, float]]:
        """计算attention转移概率"""
        trans = defaultdict(lambda: defaultdict(int))
        for clip in self.clips:
            segs = clip.get("segments", [])
            for i in range(len(segs) - 1):
                f = self._normalize_field(segs[i].get("attention_focus"), "self")
                t = self._normalize_field(segs[i+1].get("attention_focus"), "self")
                trans[f][t] += 1
        
        prob = {}
        for f, tos in trans.items():
            total = sum(tos.values())
            prob[f] = {t: c/total for t, c in tos.items()}
        return prob
    
    def infer_baseline_costs(self) -> Dict[str, float]:
        """推断不同attention下的打断代价"""
        focus_stats = defaultdict(lambda: {"total": 0, "ignored": 0})
        
        for seg in self.all_segments:
            focus = self._normalize_field(seg.get("attention_focus"), "self")
            trigger = self._normalize_field(seg.get("trigger"), "self")
            act = self._normalize_field(seg.get("speech_act"), "narrate")
            
            if trigger == "danmaku":
                focus_stats[focus]["total"] += 1
                if act != "respond":
                    focus_stats[focus]["ignored"] += 1
        
        costs = {}
        for focus, stats in focus_stats.items():
            if stats["total"] > 0:
                costs[focus] = stats["ignored"] / stats["total"]
            else:
                costs[focus] = 0.5
        return costs
    
    def extract_skeletons(self) -> List[Tuple[str, int]]:
        """提取叙事骨架"""
        skeletons = [c.get("skeleton", "") for c in self.clips if c.get("skeleton")]
        return Counter(skeletons).most_common(10)
    
    def extract_catchphrases(self, language: str = None) -> List[Tuple[str, int]]:
        """提取口癖"""
        cps = []
        for c in self.clips:
            if language and c.get("language") != language:
                continue
            cps.extend(c.get("catchphrases", []))
        return Counter(cps).most_common(20)
    
    def extract_hooks(self, language: str = None) -> List[str]:
        """提取开场示例"""
        hooks = []
        for c in self.clips:
            if language and c.get("language") != language:
                continue
            segs = c.get("segments", [])
            if segs:
                hooks.append(segs[0].get("text", "")[:100])
        return hooks[:10]
    
    def get_report(self) -> str:
        """生成分析报告"""
        lines = [
            "="*50,
            "📊 Pattern Analysis Report",
            "="*50,
            f"Total clips: {len(self.clips)}, Total segments: {len(self.all_segments)}",
        ]
        
        lines.append("\n### Attention Transitions")
        for f, tos in self.compute_attention_transitions().items():
            top = sorted(tos.items(), key=lambda x: -x[1])[:3]
            lines.append(f"- `{f}` → " + ", ".join(f"`{t}`:{p:.0%}" for t, p in top))
        
        lines.append("\n### Inferred Interruption Costs")
        for focus, cost in self.infer_baseline_costs().items():
            bar = "█" * int(cost * 10) + "░" * (10 - int(cost * 10))
            lines.append(f"- `{focus}`: {bar} {cost:.2f}")
        
        lines.append("\n### Top Catchphrases")
        cps = self.extract_catchphrases()[:10]
        lines.append(", ".join(f'"{c}"({n})' for c, n in cps))
        
        return "\n".join(lines)

### 2.2 加载标注数据

In [65]:
# 加载标注数据
DATA_PATH = PROJECT_ROOT / "data" / "annotated_clips.json"

with open(DATA_PATH, "r", encoding="utf-8") as f:
    annotated_clips = json.load(f)

print(f"✅ 加载了 {len(annotated_clips)} 个clips")

# 创建分析器
analyzer = PatternAnalyzer(annotated_clips)

# 显示报告
display(Markdown(analyzer.get_report()))

✅ 加载了 30 个clips


==================================================
📊 Pattern Analysis Report
==================================================
Total clips: 30, Total segments: 101

### Attention Transitions
- `specific` → `self`:58%, `specific`:25%, `audience`:17%
- `audience` → `audience`:40%, `self`:40%, `specific`:13%
- `self` → `self`:59%, `audience`:27%, `content`:7%
- `content` → `self`:33%, `content`:33%, `audience`:33%

### Inferred Interruption Costs
- `self`: ██████████ 1.00
- `audience`: ███░░░░░░░ 0.33
- `specific`: ██░░░░░░░░ 0.20

### Top Catchphrases
"you know"(13), "like"(11), "我觉得"(4), "对吧"(3), "you know what I mean"(3), "uhm"(3), "what the f***"(3), "对不对"(2), "我跟你们说"(2), "oh my god"(2)

In [66]:
# 查看示例数据
print("\n📝 开场 Hook 示例 (中文):")
for i, hook in enumerate(analyzer.extract_hooks("zh")[:3], 1):
    print(f"  {i}. {hook[:60]}...")

print("\n📝 叙事骨架示例:")
for skel, count in analyzer.extract_skeletons()[:5]:
    print(f"  [{count}次] {skel[:50]}...")


📝 开场 Hook 示例 (中文):
  1. 所以说你只能把这条路走完，这个就是很多时候我们会很畏首畏尾的一个原因，就是因为有这个沉没成本在这里。...
  2. 猫猫怎么了？你也想要上镜是不是，小猫猫？小猫猫你也想要上镜，那你上镜啊，小猫猫！你上镜我就给你准备上镜礼物。是陈子又冠名...
  3. 谢谢XX的SC，"也可以先从宝宝起名开始"。不行不行不行，给活物起名是我的第一线。有很多人啊，他做选择是什么，他就想赖别...

📝 叙事骨架示例:
  [1次] 共情→自我经历→对比→建议→升华...
  [1次] 互动→抱怨→对比（外面的猫）→感慨→收尾...
  [1次] 拒绝→心理分析（举例）→延伸（宠物/孩子）→总结...
  [1次] 吐槽→自我发现→担忧→自嘲→反转...
  [1次] 开场悬念→背景铺垫（穷）→核心故事（偷吃腰果）→高潮（坦白）→释放→收尾（广告）...


## Part 3: TTS 语音合成（通义千问）

包含：
- TTSClient 核心类
- Voice Design 自定义音色
- 音色缓存机制
- 测试工具

In [67]:
class TTSClient:
    """通义千问 CosyVoice TTS 客户端（增强版）"""
    
    # 📋 所有可用音色列表（从官方文档提取 2025-11-27）
    # 参考: https://help.aliyun.com/zh/model-studio/qwen-tts-realtime
    AVAILABLE_VOICES = [
        # ===== 🎀 可爱/萌系女声 =====
        "Cherry",           # 芊悦 - 阳光积极、亲切自然小姐姐
        "Serena",           # 苏瑶 - 温柔小姐姐
        "Chelsie",          # 千雪 - 二次元虚拟女友 ⭐推荐VTuber
        "Momo",             # 茉兔 - 撒娇搞怪，逗你开心 ⭐推荐VTuber
        "Vivian",           # 十三 - 拽拽的，可爱的小暴躁
        "Bunny",            # 萌小姬 - 萌属性爆棚的小萝莉 ⭐推荐VTuber
        "Bella",            # 萌宝 - 喝酒不打醉拳的小萝莉
        "Mia",              # 乖小妹 - 温顺如春水，乖巧如初雪
        "Stella",           # 少女阿月 - 甜到发腻的迷糊少女音
        "Nini",             # 声拉长了的'哥哥'，甜得骨头都酥了
        
        # ===== 👩 知性/成熟女声 =====
        "Katerina",         # 卡捷琳娜 - 御姐音色，韵律回味十足
        "Maia",             # 四月 - 知性与温柔的碰撞
        "Seren",            # 小婉 - 温和舒缓，助眠声线
        "Bellona",          # 燕铮莹 - 声音洪亮，吐字清晰，热血江湖
        "Elias",            # 墨讲师 - 学科严谨，叙事技巧
        "Jennifer",         # 詹妮弗 - 品牌级电影质感美语女声
        
        # ===== 👨 阳光/年轻男声 =====
        "Ethan",            # 晨煦 - 标准普通话，阳光温暖活力朝气
        "Ryan",             # 甜茶 - 节奏拉满，戏感炸裂
        "Aiden",            # 艾登 - 精通厨艺的美语大男孩
        "Moon",             # 月白 - 率性帅气
        "Kai",              # 凯 - 耳朵的一场SPA
        "Nofish",           # 不吃鱼 - 不会翘舌音的设计师
        
        # ===== 👴 成熟/磁性男声 =====
        "Andre",            # 安德雷 - 声音磁性，自然舒服沉稳
        "Vincent",          # 田叔 - 沙哑烟嗓，千军万马江湖豪情
        "Eldric_Sage",      # 沧明子 - 沉稳睿智老者，沧桑如松
        "Arthur",           # 徐大爷 - 质朴嗓音，满村奇闻异事
        "Neil",             # 阿闻 - 专业新闻主持人，字正腔圆
        
        # ===== 🌍 外语/方言特色 =====
        "Dolce",            # 多尔切 - 慵懒意大利大叔（男）
        "Sohee",            # 素熙 - 温柔开朗韩国欧尼（女）
        "Ono_Anna",         # 小野杏 - 鬼灵精怪青梅竹马（女，日系）
        "Lenn",             # 莱恩 - 叛逆德国青年（男）
        "Emilien",          # 埃米尔安 - 浪漫法国大哥哥（男）
        "Li",               # 南京老李 - 耐心瑜伽老师（男，南京话）
        "Marcus",           # 秦川 - 老陕味道（男，陕西话）
        "Roy",              # 阿杰 - 诙谐直爽台湾哥仔（男，闽南语）
        "Peter",            # 李彼得 - 天津相声捧哏（男，天津话）
        "Sunny",            # 晴儿 - 甜到心里的川妹子（女，四川话）
        "Eric",             # 程川 - 跳脱市井成都男子（男，四川话）
        
        # ===== 👶 儿童/特殊 =====
        "Mochi",            # 沙小弥 - 聪明伶俐小大人，童真早慧（男童）
        "Pip",              # 顽屁小孩 - 调皮捣蛋充满童真（男童）
        "Ebona",            # 诡婆婆 - 幽暗角落，童年阴影恐惧（特殊）
    ]
    
    # 🎯 VTuber推荐音色
    VTUBER_RECOMMENDED = ["Chelsie", "Momo", "Bunny", "Cherry", "Vivian", "Stella"]
    
    def __init__(self):
        self.enabled = False
        self.synthesizer_class = None
        self.audio_format = None
        
        # 情绪控制参数
        self.rate = 1.0      # 语速 0.5-2.0
        self.pitch = 0       # 音调 -500到500
        self.volume = 50     # 音量 0-100
        
        # 录制缓存
        self.recording_buffer = []
        
        # Voice Design 音色缓存（避免重复创建）
        # 格式: {(emotion, base_voice, intensity): voice_name}
        self.voice_cache = {}
        
        # 音色配置文件路径
        self.voice_cache_file = PROJECT_ROOT / "output" / "voice_cache.json"
        
        # 加载已保存的音色缓存
        self._load_voice_cache()
        
        api_key = os.getenv("DASHSCOPE_API_KEY")
        if api_key:
            try:
                import dashscope
                from dashscope.audio.tts_v2 import SpeechSynthesizer, AudioFormat
                
                dashscope.api_key = api_key
                self.synthesizer_class = SpeechSynthesizer
                self.audio_format = AudioFormat.MP3_22050HZ_MONO_256KBPS
                self.model = os.getenv("TTS_MODEL", "cosyvoice-v3-flash")
                self.voice = os.getenv("TTS_VOICE", "Bella")  # 默认音色改为Bella（萌宝 - 喝酒不打醉拳的小萝莉）
                self.enabled = True
                print(f"✅ TTS 已启用: model={self.model}, voice={self.voice}")
                print(f"   情绪参数: rate={self.rate}, pitch={self.pitch}, volume={self.volume}")
                print(f"   可用音色: {len(self.AVAILABLE_VOICES)} 个")
            except ImportError:
                print("⚠️ dashscope 未安装")
        else:
            print("⚠️ 未设置 DASHSCOPE_API_KEY，TTS 已禁用")
    
    def synthesize(self, text: str, emotion_boost: float = 0.0) -> Optional[bytes]:
        """
        合成语音，返回音频数据（使用 QwenTtsRealtime API）
        
        Args:
            text: 要合成的文本
            emotion_boost: 情绪增强系数 0.0-1.0（暂未实现，保留接口）
                          0.0 = 正常语速
                          1.0 = 最大情绪变化（语速±50%，音调±200）
        
        Returns:
            MP3格式的音频数据（bytes），失败返回None
        """
        # #region agent log
        import json, time, os
        log_path = r"d:\vtuberclip\echuu-agent\.cursor\debug.log"
        try:
            os.makedirs(os.path.dirname(log_path), exist_ok=True)
            with open(log_path, "a", encoding="utf-8") as f:
                f.write(json.dumps({"sessionId":"debug-session","runId":"run1","hypothesisId":"A","location":"TTSClient.synthesize:entry","message":"synthesize called","data":{"text_len":len(text),"emotion_boost":emotion_boost,"voice":self.voice,"enabled":self.enabled},"timestamp":int(time.time()*1000)})+"\n")
        except Exception as log_err:
            print(f"[DEBUG] Log error: {log_err}")
        # #endregion
        
        if not self.enabled:
            return None
        
        try:
            from dashscope.audio.qwen_tts_realtime import QwenTtsRealtime, QwenTtsRealtimeCallback, AudioFormat
            from pydub import AudioSegment
            
            # #region agent log
            try:
                with open(log_path, "a", encoding="utf-8") as f:
                    f.write(json.dumps({"sessionId":"debug-session","runId":"run1","hypothesisId":"B","location":"TTSClient.synthesize:before_realtime","message":"before QwenTtsRealtime init","data":{"voice":self.voice,"model":"qwen3-tts-flash-realtime"},"timestamp":int(time.time()*1000)})+"\n")
            except: pass
            # #endregion
            
            # 使用 qwen3-tts-flash-realtime 模型（支持预设音色如 Bella）
            model = 'qwen3-tts-flash-realtime'
            voice_name = self.voice
            
            # 创建回调收集音频
            audio_chunks = []
            complete_event = threading.Event()
            
            class TempCallback(QwenTtsRealtimeCallback):
                def on_event(self, response):
                    # #region agent log
                    try:
                        with open(log_path, "a", encoding="utf-8") as f:
                            f.write(json.dumps({"sessionId":"debug-session","runId":"run1","hypothesisId":"C","location":"TempCallback.on_event","message":"callback event","data":{"type":response.get('type')},"timestamp":int(time.time()*1000)})+"\n")
                    except: pass
                    # #endregion
                    
                    if response.get('type') == 'response.audio.delta':
                        audio_chunks.append(base64.b64decode(response['delta']))
                    elif response.get('type') == 'session.finished':
                        complete_event.set()
            
            callback = TempCallback()
            
            # #region agent log
            try:
                with open(log_path, "a", encoding="utf-8") as f:
                    f.write(json.dumps({"sessionId":"debug-session","runId":"run1","hypothesisId":"D","location":"TTSClient.synthesize:before_connect","message":"before connect","data":{"model":model,"voice":voice_name},"timestamp":int(time.time()*1000)})+"\n")
            except: pass
            # #endregion
            
            qwen_tts = QwenTtsRealtime(
                model=model,
                callback=callback,
                url='wss://dashscope.aliyuncs.com/api-ws/v1/realtime'
            )
            
            qwen_tts.connect()
            
            # #region agent log
            try:
                with open(log_path, "a", encoding="utf-8") as f:
                    f.write(json.dumps({"sessionId":"debug-session","runId":"run1","hypothesisId":"E","location":"TTSClient.synthesize:after_connect","message":"after connect","data":{},"timestamp":int(time.time()*1000)})+"\n")
            except: pass
            # #endregion
            
            # 设置音色和参数（server_commit 模式）
            qwen_tts.update_session(
                voice=voice_name,
                response_format=AudioFormat.PCM_24000HZ_MONO_16BIT,
                mode='server_commit'
            )
            
            # #region agent log
            try:
                with open(log_path, "a", encoding="utf-8") as f:
                    f.write(json.dumps({"sessionId":"debug-session","runId":"run1","hypothesisId":"F","location":"TTSClient.synthesize:after_update_session","message":"after update_session","data":{"voice":voice_name},"timestamp":int(time.time()*1000)})+"\n")
            except: pass
            # #endregion
            
            # 发送文本
            qwen_tts.append_text(text)
            qwen_tts.finish()
            
            # 等待完成（最多30秒）
            complete_event.wait(timeout=30)
            qwen_tts.close()
            
            # #region agent log
            try:
                with open(log_path, "a", encoding="utf-8") as f:
                    f.write(json.dumps({"sessionId":"debug-session","runId":"run1","hypothesisId":"G","location":"TTSClient.synthesize:after_finish","message":"after finish","data":{"audio_chunks_count":len(audio_chunks)},"timestamp":int(time.time()*1000)})+"\n")
            except: pass
            # #endregion
            
            if audio_chunks:
                # 将PCM转换为MP3
                pcm_data = b''.join(audio_chunks)
                audio = AudioSegment(
                    pcm_data,
                    frame_rate=24000,
                    channels=1,
                    sample_width=2
                )
                mp3_buffer = io.BytesIO()
                audio.export(mp3_buffer, format="mp3")
                audio_data = mp3_buffer.getvalue()
                
                # #region agent log
                try:
                    with open(log_path, "a", encoding="utf-8") as f:
                        f.write(json.dumps({"sessionId":"debug-session","runId":"run1","hypothesisId":"H","location":"TTSClient.synthesize:success","message":"synthesize success","data":{"audio_size":len(audio_data)},"timestamp":int(time.time()*1000)})+"\n")
                except: pass
                # #endregion
                
                # 添加到录制缓存
                if audio_data:
                    self.recording_buffer.append(audio_data)
                
                return audio_data
            else:
                # #region agent log
                try:
                    with open(log_path, "a", encoding="utf-8") as f:
                        f.write(json.dumps({"sessionId":"debug-session","runId":"run1","hypothesisId":"I","location":"TTSClient.synthesize:no_audio","message":"no audio chunks received","data":{},"timestamp":int(time.time()*1000)})+"\n")
                except: pass
                # #endregion
                print("[TTS] 未收到音频数据")
                return None
                
        except Exception as e:
            # #region agent log
            try:
                with open(log_path, "a", encoding="utf-8") as f:
                    f.write(json.dumps({"sessionId":"debug-session","runId":"run1","hypothesisId":"J","location":"TTSClient.synthesize:error","message":"synthesize error","data":{"error":str(e),"error_type":type(e).__name__},"timestamp":int(time.time()*1000)})+"\n")
            except: pass
            # #endregion
            print(f"[TTS] 错误: {e}")
            import traceback
            traceback.print_exc()
            return None
    
    def change_voice(self, voice: str):
        """切换音色"""
        if voice in self.AVAILABLE_VOICES:
            self.voice = voice
            print(f"🔊 音色已切换为: {voice}")
        else:
            print(f"⚠️ 未知音色: {voice}")
            print(f"   可用音色: {', '.join(self.AVAILABLE_VOICES[:5])}...")
    
    def set_emotion_params(self, rate: float = None, pitch: int = None, volume: int = None):
        """设置情绪参数"""
        if rate is not None:
            self.rate = max(0.5, min(2.0, rate))
        if pitch is not None:
            self.pitch = max(-500, min(500, pitch))
        if volume is not None:
            self.volume = max(0, min(100, volume))
        print(f"🎛️ 情绪参数已更新: rate={self.rate}, pitch={self.pitch}, volume={self.volume}")
    
    def start_recording(self):
        """开始录制"""
        self.recording_buffer = []
        print("🔴 开始录制直播音频...")
    
    def save_recording(self, output_path: str):
        """保存录制的音频"""
        if not self.recording_buffer:
            print("⚠️ 没有可保存的音频")
            return
        
        try:
            from pydub import AudioSegment
            import io
            
            # 合并所有音频片段
            combined = AudioSegment.empty()
            for audio_data in self.recording_buffer:
                audio = AudioSegment.from_file(io.BytesIO(audio_data), format="mp3")
                combined += audio
            
            # 保存
            combined.export(output_path, format="mp3")
            print(f"💾 直播音频已保存到: {output_path}")
            print(f"   总时长: {len(combined)/1000:.1f}秒")
        except ImportError:
            print("⚠️ 需要安装 pydub: pip install pydub")
            # 简单保存：拼接原始数据（不推荐）
            with open(output_path, "wb") as f:
                for audio_data in self.recording_buffer:
                    f.write(audio_data)
            print(f"💾 音频已保存（未合并）: {output_path}")
        except Exception as e:
            print(f"❌ 保存失败: {e}")
    
    # ==================== Voice Design 功能 ====================
    
    def _load_voice_cache(self):
        """从文件加载音色缓存"""
        if self.voice_cache_file.exists():
            try:
                with open(self.voice_cache_file, "r", encoding="utf-8") as f:
                    self.voice_cache = json.load(f)
                print(f"✅ 已加载 {len(self.voice_cache)} 个已保存的音色")
            except Exception as e:
                print(f"⚠️ 加载音色缓存失败: {e}")
                self.voice_cache = {}
        else:
            self.voice_cache = {}
    
    def _save_voice_cache(self):
        """保存音色缓存到文件"""
        try:
            self.voice_cache_file.parent.mkdir(parents=True, exist_ok=True)
            with open(self.voice_cache_file, "w", encoding="utf-8") as f:
                json.dump(self.voice_cache, f, ensure_ascii=False, indent=2)
        except Exception as e:
            print(f"⚠️ 保存音色缓存失败: {e}")
    
    def get_voice_cache_key(self, emotion: str, base_voice: str, intensity: float) -> str:
        """生成音色缓存的键"""
        # 将intensity四舍五入到1位小数，避免浮点数精度问题
        intensity_rounded = round(intensity, 1)
        # 将小数点替换为下划线，确保preferred_name只包含字母、数字和下划线
        intensity_str = str(intensity_rounded).replace('.', '_')
        return f"{base_voice}_{emotion}_{intensity_str}"
    
    def list_saved_voices(self) -> Dict[str, str]:
        """列出所有已保存的音色"""
        return self.voice_cache.copy()
    
    def get_or_create_voice(self,
                           emotion: str = "neutral",
                           base_voice: str = "Bella",
                           intensity: float = 0.5,
                           force_create: bool = False) -> Optional[str]:
        """
        获取或创建音色（带缓存）
        
        Args:
            emotion: 情绪类型
            base_voice: 基础音色
            intensity: 情绪强度
            force_create: 是否强制重新创建（忽略缓存）
        
        Returns:
            音色名称，失败返回None
        """
        cache_key = self.get_voice_cache_key(emotion, base_voice, intensity)
        
        # 检查缓存
        if not force_create and cache_key in self.voice_cache:
            voice_name = self.voice_cache[cache_key]
            print(f"✅ 使用已保存的音色: {voice_name} (缓存键: {cache_key})")
            return voice_name
        
        # 创建新音色
        print(f"🎨 创建新音色 (缓存键: {cache_key})...")
        voice_name = self.create_voice_with_emotion(
            emotion=emotion,
            base_voice=base_voice,
            intensity=intensity,
            preferred_name=cache_key  # 使用缓存键作为音色名称
        )
        
        if voice_name:
            # 保存到缓存
            self.voice_cache[cache_key] = voice_name
            self._save_voice_cache()
            print(f"💾 音色已保存到缓存: {cache_key} -> {voice_name}")
        
        return voice_name
    
    def generate_emotion_prompt(self, 
                                emotion: str = "neutral",
                                base_voice: str = "Bella",
                                intensity: float = 0.5,
                                additional_attributes: str = "") -> str:
        """
        生成情绪描述提示词（voice_prompt）
        
        Args:
            emotion: 情绪类型 (neutral, happy, sad, excited, angry, gentle, serious, etc.)
            base_voice: 基础音色描述 (默认Bella: 萌宝小萝莉)
            intensity: 情绪强度 0.0-1.0
            additional_attributes: 额外的声学属性描述
        
        Returns:
            完整的voice_prompt描述文本
        """
        # 基础音色描述
        base_descriptions = {
            "Bella": "萌宝小萝莉，音色稚嫩可爱",
            "Cherry": "阳光积极、亲切自然的小姐姐",
            "Ethan": "标准普通话，阳光温暖的男性声音",
            "Serena": "温柔的小姐姐",
        }
        base_desc = base_descriptions.get(base_voice, "年轻女性声音")
        
        # 情绪描述映射
        emotion_descriptions = {
            "neutral": "平静自然",
            "happy": "开心愉快，语调上扬，充满活力",
            "sad": "悲伤低沉，语速偏慢，情绪浓烈，带有哭腔，语调哀怨",
            "excited": "兴奋激动，语调不断上扬，快速而充满活力的节奏，音量响亮",
            "angry": "愤怒严厉，音量增大，语速加快，语调强硬",
            "gentle": "温柔舒缓，语速适中，音调柔和",
            "serious": "严肃认真，语调平稳，发音清晰精准",
            "playful": "调皮活泼，音调偏高且起伏明显，营造出黏人、做作的听觉效果",
            "enthusiastic": "热情洋溢，高亢的嗓音，语调随兴奋情绪不断上扬",
            "crying": "极度悲伤，带着明显哭腔，声音沙哑，语速缓慢，情感强烈",
        }
        
        emotion_desc = emotion_descriptions.get(emotion.lower(), emotion)
        
        # 根据强度调整描述
        if intensity > 0.7:
            intensity_modifier = "非常"
        elif intensity > 0.4:
            intensity_modifier = ""
        else:
            intensity_modifier = "略微"
        
        # 构建完整描述
        prompt_parts = [f"{base_desc}"]
        
        if emotion != "neutral":
            prompt_parts.append(f"{intensity_modifier}{emotion_desc}")
        
        if additional_attributes:
            prompt_parts.append(additional_attributes)
        
        # 添加通用描述
        prompt_parts.append("发音清晰，表达自然流畅")
        
        full_prompt = "，".join(prompt_parts) + "。"
        
        return full_prompt
    
    def create_voice_with_emotion(self,
                                  emotion: str = "neutral",
                                  base_voice: str = "Bella",
                                  intensity: float = 0.5,
                                  preview_text: str = "你好，我是AI主播，今天来给大家讲个有趣的故事！",
                                  preferred_name: str = None,
                                  language: str = "zh",
                                  additional_attributes: str = "") -> Optional[str]:
        """
        使用Voice Design API创建带情绪的自定义音色
        
        Args:
            emotion: 情绪类型
            base_voice: 基础音色
            intensity: 情绪强度 0.0-1.0
            preview_text: 预览文本
            preferred_name: 音色名称（可选）
            language: 语言代码 (zh/en)
            additional_attributes: 额外的声学属性描述
        
        Returns:
            创建的音色名称，失败返回None
        """
        if not self.enabled:
            print("⚠️ TTS 未启用")
            return None
        
        try:
            import requests
            import base64
            
            api_key = os.getenv("DASHSCOPE_API_KEY")
            if not api_key:
                print("⚠️ 未设置 DASHSCOPE_API_KEY")
                return None
            
            # 生成voice_prompt
            voice_prompt = self.generate_emotion_prompt(
                emotion=emotion,
                base_voice=base_voice,
                intensity=intensity,
                additional_attributes=additional_attributes
            )
            
            # 生成音色名称（确保符合API要求：只包含字母、数字、下划线，最大16字符）
            if not preferred_name:
                # 使用缓存键格式，但需要清理以确保符合API要求
                cache_key = self.get_voice_cache_key(emotion, base_voice, intensity)
                preferred_name = cache_key
            else:
                # 清理preferred_name：只保留字母、数字、下划线
                import re
                preferred_name = re.sub(r'[^a-zA-Z0-9_]', '_', preferred_name)
            
            # 确保长度不超过16字符
            if len(preferred_name) > 16:
                preferred_name = preferred_name[:16]
            
            # API端点（根据地域选择）
            # 北京地域: https://dashscope.aliyuncs.com/api/v1/services/audio/tts/customization
            # 新加坡地域: https://dashscope-intl.aliyuncs.com/api/v1/services/audio/tts/customization
            url = os.getenv("DASHSCOPE_URL", "https://dashscope.aliyuncs.com/api/v1/services/audio/tts/customization")
            if "intl" not in url and "dashscope.aliyuncs.com" in url:
                # 默认使用北京地域
                pass
            
            headers = {
                "Authorization": f"Bearer {api_key}",
                "Content-Type": "application/json"
            }
            
            data = {
                "model": "qwen-voice-design",
                "input": {
                    "action": "create",
                    "target_model": "qwen3-tts-vd-realtime-2025-12-16",
                    "voice_prompt": voice_prompt,
                    "preview_text": preview_text,
                    "preferred_name": preferred_name,
                    "language": language
                },
                "parameters": {
                    "sample_rate": 24000,
                    "response_format": "wav"
                }
            }
            
            print(f"🎨 正在创建自定义音色...")
            print(f"   情绪: {emotion} (强度: {intensity:.1f})")
            print(f"   描述: {voice_prompt[:50]}...")
            
            response = requests.post(url, headers=headers, json=data, timeout=60)
            
            if response.status_code == 200:
                result = response.json()
                voice_name = result["output"]["voice"]
                
                # 保存预览音频（可选）
                if "preview_audio" in result["output"]:
                    base64_audio = result["output"]["preview_audio"]["data"]
                    audio_bytes = base64.b64decode(base64_audio)
                    preview_filename = f"{voice_name}_preview.wav"
                    with open(preview_filename, "wb") as f:
                        f.write(audio_bytes)
                    print(f"✅ 音色创建成功: {voice_name}")
                    print(f"   预览音频已保存: {preview_filename}")
                else:
                    print(f"✅ 音色创建成功: {voice_name}")
                
                return voice_name
            else:
                print(f"❌ 创建音色失败: {response.status_code}")
                print(f"   错误信息: {response.text}")
                return None
                
        except Exception as e:
            print(f"❌ 创建音色时出错: {e}")
            import traceback
            traceback.print_exc()
            return None
    
    def synthesize_with_emotion(self,
                     text: str,
                     emotion: str = "neutral",
                     base_voice: str = "Bella",
                     intensity: float = 0.5,
                     use_voice_design: bool = True,
                     additional_attributes: str = "") -> Optional[bytes]:
        """
        使用指定情绪合成语音
        
        Args:
            text: 要合成的文本
            emotion: 情绪类型
            base_voice: 基础音色
            intensity: 情绪强度
            use_voice_design: 是否使用Voice Design API（True）或使用系统音色+emotion_boost（False）
            additional_attributes: 额外的声学属性描述
        
        Returns:
            音频数据
        """
        if not self.enabled:
            return None
        
        if use_voice_design:
            # 使用Voice Design获取或创建自定义音色（带缓存）
            voice_name = self.get_or_create_voice(
                emotion=emotion,
                base_voice=base_voice,
                intensity=intensity,
                force_create=False  # 使用缓存，避免重复创建
            )
            
            if voice_name:
                # 使用创建的自定义音色进行合成
                # 注意：需要使用qwen3-tts-vd-realtime-2025-12-16模型
                try:
                    from dashscope.audio.qwen_tts_realtime import QwenTtsRealtime, AudioFormat
                    import threading
                    import time
                    import base64
                    
                    # 创建临时回调来收集音频
                    audio_chunks = []
                    complete_event = threading.Event()
                    
                    class TempCallback:
                        def on_event(self, response):
                            if response.get('type') == 'response.audio.delta':
                                audio_chunks.append(base64.b64decode(response['delta']))
                            elif response.get('type') == 'session.finished':
                                complete_event.set()
                    
                    callback = TempCallback()
                    qwen_tts = QwenTtsRealtime(
                        model="qwen3-tts-vd-realtime-2025-12-16",
                        callback=callback,
                        url='wss://dashscope.aliyuncs.com/api-ws/v1/realtime'
                    )
                    
                    qwen_tts.connect()
                    qwen_tts.update_session(
                        voice=voice_name,
                        response_format=AudioFormat.PCM_24000HZ_MONO_16BIT,
                        mode='server_commit'
                    )
                    
                    qwen_tts.append_text(text)
                    qwen_tts.finish()
                    complete_event.wait(timeout=30)
                    qwen_tts.close()
                    
                    if audio_chunks:
                        # 将PCM转换为MP3（需要pydub）
                        try:
                            from pydub import AudioSegment
                            import io
                            import numpy as np
                            
                            # 合并PCM数据
                            pcm_data = b''.join(audio_chunks)
                            # 转换为AudioSegment
                            audio = AudioSegment(
                                pcm_data,
                                frame_rate=24000,
                                channels=1,
                                sample_width=2
                            )
                            # 转换为MP3
                            mp3_buffer = io.BytesIO()
                            audio.export(mp3_buffer, format="mp3")
                            audio_data = mp3_buffer.getvalue()
                            
                            if audio_data:
                                self.recording_buffer.append(audio_data)
                            return audio_data
                        except ImportError:
                            print("⚠️ 需要安装 pydub 和 numpy 来转换音频格式")
                            return None
                    else:
                        print("⚠️ 未收到音频数据")
                        return None
                        
                except Exception as e:
                    print(f"❌ 使用自定义音色合成失败: {e}")
                    # 降级到使用系统音色
                    return self.synthesize(text, emotion_boost=intensity)
            else:
                # Voice Design创建失败，降级到系统音色
                print("⚠️ Voice Design创建失败，使用系统音色")
                return self.synthesize(text, emotion_boost=intensity)
        else:
            # 使用系统音色 + emotion_boost
            return self.synthesize(text, emotion_boost=intensity)


# 初始化 TTS
tts = TTSClient()

✅ 已加载 1 个已保存的音色
✅ TTS 已启用: model=cosyvoice-v3-flash, voice=Bella
   情绪参数: rate=1.0, pitch=0, volume=50
   可用音色: 41 个


In [68]:
# ==================== Voice Design 功能演示 ====================

print("🎨 Voice Design 功能已添加！")
print("\n📚 主要功能：")
print("1. generate_emotion_prompt() - 生成情绪描述提示词")
print("2. create_voice_with_emotion() - 创建带情绪的自定义音色")
print("3. synthesize_with_emotion() - 使用指定情绪合成语音")
print("\n💡 使用示例：")
print("""
# 示例1: 生成情绪描述
prompt = tts.generate_emotion_prompt(
    emotion="sad",
    base_voice="Bella",
    intensity=0.8,
    additional_attributes="音高起伏大，带有明显的颤抖"
)
print(prompt)

# 示例2: 创建带情绪的音色
voice_name = tts.create_voice_with_emotion(
    emotion="excited",
    base_voice="Bella",
    intensity=0.9,
    preview_text="天哪！太好笑了！真的超级开心！"
)

# 示例3: 直接使用情绪合成
audio = tts.synthesize_with_emotion(
    text="今天真是太开心了！",
    emotion="happy",
    base_voice="Bella",
    intensity=0.7,
    use_voice_design=True  # 使用Voice Design API
)
if audio:
    display(Audio(audio, autoplay=False))
""")

print("\n🎭 支持的情绪类型：")
emotions = ["neutral", "happy", "sad", "excited", "angry", "gentle", 
            "serious", "playful", "enthusiastic", "crying"]
for i, emo in enumerate(emotions, 1):
    print(f"  {i:2d}. {emo}")

print("\n⚙️ 配置说明：")
print("1. API URL配置（根据你的API Key地域选择）：")
print("   - 北京地域: 默认使用 https://dashscope.aliyuncs.com")
print("   - 新加坡地域: 设置环境变量 DASHSCOPE_URL=https://dashscope-intl.aliyuncs.com/api/v1/services/audio/tts/customization")
print("   - 或在代码中直接修改 create_voice_with_emotion() 方法中的 url 变量")

print("\n⚠️ 注意事项：")
print("1. Voice Design API需要 qwen3-tts-vd-realtime-2025-12-16 模型")
print("2. 创建音色需要调用API，每次创建会产生费用（$0.2/次）")
print("3. ✅ 音色缓存功能：使用 get_or_create_voice() 会自动缓存，避免重复创建")
print("4. ✅ 缓存持久化：音色缓存保存在 output/voice_cache.json，重启后仍然有效")
print("5. 如果Voice Design失败，会自动降级到系统音色+emotion_boost模式")
print("6. 需要安装: pip install requests (用于API调用)")
print("7. 使用自定义音色合成时，需要安装: pip install pydub numpy (用于音频格式转换)")
print("\n💾 音色缓存使用方法：")
print("  # 推荐方式：使用 get_or_create_voice()（自动缓存）")
print("  voice_name = tts.get_or_create_voice(emotion='happy', base_voice='Bella', intensity=0.7)")
print("  # 第一次调用会创建并缓存，后续调用直接使用缓存")

🎨 Voice Design 功能已添加！

📚 主要功能：
1. generate_emotion_prompt() - 生成情绪描述提示词
2. create_voice_with_emotion() - 创建带情绪的自定义音色
3. synthesize_with_emotion() - 使用指定情绪合成语音

💡 使用示例：

# 示例1: 生成情绪描述
prompt = tts.generate_emotion_prompt(
    emotion="sad",
    base_voice="Bella",
    intensity=0.8,
    additional_attributes="音高起伏大，带有明显的颤抖"
)
print(prompt)

# 示例2: 创建带情绪的音色
voice_name = tts.create_voice_with_emotion(
    emotion="excited",
    base_voice="Bella",
    intensity=0.9,
    preview_text="天哪！太好笑了！真的超级开心！"
)

# 示例3: 直接使用情绪合成
audio = tts.synthesize_with_emotion(
    text="今天真是太开心了！",
    emotion="happy",
    base_voice="Bella",
    intensity=0.7,
    use_voice_design=True  # 使用Voice Design API
)
if audio:
    display(Audio(audio, autoplay=False))


🎭 支持的情绪类型：
   1. neutral
   2. happy
   3. sad
   4. excited
   5. angry
   6. gentle
   7. serious
   8. playful
   9. enthusiastic
  10. crying

⚙️ 配置说明：
1. API URL配置（根据你的API Key地域选择）：
   - 北京地域: 默认使用 https://dashscope.aliyuncs.com
   - 新加坡地域: 设置环境变

In [69]:
# ==================== Voice Design 测试示例 ====================

if tts.enabled:
    print("🧪 测试 Voice Design 功能\n")
    
    # 测试1: 生成情绪描述
    print("=" * 60)
    print("测试1: 生成不同情绪的描述提示词")
    print("=" * 60)
    
    test_emotions = [
        ("happy", 0.8, "开心愉快"),
        ("sad", 0.7, "悲伤"),
        ("excited", 0.9, "兴奋激动"),
    ]
    
    for emotion, intensity, desc in test_emotions:
        prompt = tts.generate_emotion_prompt(
            emotion=emotion,
            base_voice="Bella",
            intensity=intensity
        )
        print(f"\n【{desc}】 (强度: {intensity})")
        print(f"  描述: {prompt}")
    
    print("\n" + "=" * 60)
    print("💡 提示：")
    print("1. 要实际创建音色，请运行: voice_name = tts.create_voice_with_emotion(...)")
    print("2. 要直接合成带情绪的语音，请运行: audio = tts.synthesize_with_emotion(...)")
    print("3. 注意：Voice Design API调用会产生费用")
    print("=" * 60)
else:
    print("⚠️ TTS 未启用，无法测试")

🧪 测试 Voice Design 功能

测试1: 生成不同情绪的描述提示词

【开心愉快】 (强度: 0.8)
  描述: 萌宝小萝莉，音色稚嫩可爱，非常开心愉快，语调上扬，充满活力，发音清晰，表达自然流畅。

【悲伤】 (强度: 0.7)
  描述: 萌宝小萝莉，音色稚嫩可爱，悲伤低沉，语速偏慢，情绪浓烈，带有哭腔，语调哀怨，发音清晰，表达自然流畅。

【兴奋激动】 (强度: 0.9)
  描述: 萌宝小萝莉，音色稚嫩可爱，非常兴奋激动，语调不断上扬，快速而充满活力的节奏，音量响亮，发音清晰，表达自然流畅。

💡 提示：
1. 要实际创建音色，请运行: voice_name = tts.create_voice_with_emotion(...)
2. 要直接合成带情绪的语音，请运行: audio = tts.synthesize_with_emotion(...)
3. 注意：Voice Design API调用会产生费用


In [70]:
# ==================== 音色缓存使用示例 ====================

print("💾 音色缓存功能说明\n")
print("=" * 60)
print("📚 主要功能：")
print("1. get_or_create_voice() - 获取或创建音色（自动缓存）")
print("2. list_saved_voices() - 查看所有已保存的音色")
print("3. 自动保存到文件: output/voice_cache.json")
print("\n💡 使用示例：")
print("""
# 方式1: 使用 get_or_create_voice()（推荐，自动缓存）
voice_name = tts.get_or_create_voice(
    emotion="happy",
    base_voice="Bella",
    intensity=0.7
)
# 第一次调用会创建音色并保存
# 后续调用会直接使用缓存，不会重复创建

# 方式2: 手动创建并保存
voice_name = tts.create_voice_with_emotion(
    emotion="sad",
    base_voice="Bella",
    intensity=0.8
)
# 创建后会自动保存到缓存

# 方式3: 查看所有已保存的音色
saved_voices = tts.list_saved_voices()
for cache_key, voice_name in saved_voices.items():
    print(f"{cache_key} -> {voice_name}")

# 方式4: 强制重新创建（忽略缓存）
voice_name = tts.get_or_create_voice(
    emotion="excited",
    base_voice="Bella",
    intensity=0.9,
    force_create=True  # 强制重新创建
)
""")

print("\n🎯 最佳实践：")
print("1. 首次使用时，调用 get_or_create_voice() 创建音色")
print("2. 后续使用相同参数时，会自动从缓存读取，无需重复创建")
print("3. 音色缓存保存在 output/voice_cache.json，重启后仍然有效")
print("4. 使用 synthesize_with_emotion() 时，会自动使用缓存机制")
print("=" * 60)

# 显示当前已保存的音色
# 检查voice_cache属性是否存在（兼容旧版本的TTSClient）
if not hasattr(tts, 'voice_cache'):
    print("\n⚠️ TTSClient实例需要重新初始化以支持音色缓存功能")
    print("   请重新运行: tts = TTSClient()")
    print("   或者手动初始化: tts.voice_cache = {}")
    # 尝试初始化（如果可能）
    try:
        tts.voice_cache = {}
        tts.voice_cache_file = PROJECT_ROOT / "output" / "voice_cache.json"
        if hasattr(tts, '_load_voice_cache'):
            tts._load_voice_cache()
        print("   ✅ 已自动初始化voice_cache")
    except:
        pass

if hasattr(tts, 'voice_cache') and tts.voice_cache:
    print(f"\n📋 当前已保存的音色 ({len(tts.voice_cache)} 个):")
    for cache_key, voice_name in list(tts.voice_cache.items())[:5]:
        print(f"  • {cache_key} -> {voice_name}")
    if len(tts.voice_cache) > 5:
        remaining = len(tts.voice_cache) - 5
        print(f"  ... 还有 {remaining} 个")
elif hasattr(tts, 'voice_cache'):
    print("\n📋 当前没有已保存的音色")

💾 音色缓存功能说明

📚 主要功能：
1. get_or_create_voice() - 获取或创建音色（自动缓存）
2. list_saved_voices() - 查看所有已保存的音色
3. 自动保存到文件: output/voice_cache.json

💡 使用示例：

# 方式1: 使用 get_or_create_voice()（推荐，自动缓存）
voice_name = tts.get_or_create_voice(
    emotion="happy",
    base_voice="Bella",
    intensity=0.7
)
# 第一次调用会创建音色并保存
# 后续调用会直接使用缓存，不会重复创建

# 方式2: 手动创建并保存
voice_name = tts.create_voice_with_emotion(
    emotion="sad",
    base_voice="Bella",
    intensity=0.8
)
# 创建后会自动保存到缓存

# 方式3: 查看所有已保存的音色
saved_voices = tts.list_saved_voices()
for cache_key, voice_name in saved_voices.items():
    print(f"{cache_key} -> {voice_name}")

# 方式4: 强制重新创建（忽略缓存）
voice_name = tts.get_or_create_voice(
    emotion="excited",
    base_voice="Bella",
    intensity=0.9,
    force_create=True  # 强制重新创建
)


🎯 最佳实践：
1. 首次使用时，调用 get_or_create_voice() 创建音色
2. 后续使用相同参数时，会自动从缓存读取，无需重复创建
3. 音色缓存保存在 output/voice_cache.json，重启后仍然有效
4. 使用 synthesize_with_emotion() 时，会自动使用缓存机制

📋 当前已保存的音色 (1 个):
  • Bella_happy_0_7 -> qwen-tts-vd-Bella_happ

In [71]:
# ==================== 实际使用示例 ====================

# 检查voice_cache属性是否存在（兼容旧版本的TTSClient）
if not hasattr(tts, 'voice_cache'):
    print("⚠️ TTSClient实例需要重新初始化以支持音色缓存功能")
    print("   请重新运行: tts = TTSClient()")
    # 尝试初始化（如果可能）
    try:
        tts.voice_cache = {}
        tts.voice_cache_file = PROJECT_ROOT / "output" / "voice_cache.json"
        if hasattr(tts, '_load_voice_cache'):
            tts._load_voice_cache()
        print("   ✅ 已自动初始化voice_cache")
    except Exception as e:
        print(f"   ❌ 自动初始化失败: {e}")
        print("   请重新运行: tts = TTSClient()")

if tts.enabled and hasattr(tts, 'voice_cache'):
    print("🧪 演示：创建并缓存音色\n")
    
    # 示例1: 第一次创建（会调用API）
    print("=" * 60)
    print("示例1: 创建 'happy' 情绪音色（首次创建）")
    print("=" * 60)
    voice_name_1 = tts.get_or_create_voice(
        emotion="happy",
        base_voice="Bella",
        intensity=0.7
    )
    print(f"返回的音色名称: {voice_name_1}\n")
    
    # 示例2: 再次使用相同参数（会使用缓存，不调用API）
    print("=" * 60)
    print("示例2: 再次使用相同参数（使用缓存）")
    print("=" * 60)
    voice_name_2 = tts.get_or_create_voice(
        emotion="happy",
        base_voice="Bella",
        intensity=0.7
    )
    print(f"返回的音色名称: {voice_name_2}")
    print(f"是否相同: {voice_name_1 == voice_name_2}\n")
    
    # 示例3: 查看所有已保存的音色
    print("=" * 60)
    print("示例3: 查看所有已保存的音色")
    print("=" * 60)
    saved_voices = tts.list_saved_voices()
    if saved_voices:
        for cache_key, voice_name in saved_voices.items():
            print(f"  {cache_key} -> {voice_name}")
    else:
        print("  暂无已保存的音色")
    
    print("\n" + "=" * 60)
    print("💡 提示：")
    print("1. 首次创建音色会调用API并产生费用（$0.2/次）")
    print("2. 后续使用相同参数会直接使用缓存，无需再次调用API")
    print("3. 音色缓存保存在: output/voice_cache.json")
    print("4. 重启notebook后，缓存仍然有效")
    print("=" * 60)
else:
    print("⚠️ TTS 未启用，无法演示")

🧪 演示：创建并缓存音色

示例1: 创建 'happy' 情绪音色（首次创建）
✅ 使用已保存的音色: qwen-tts-vd-Bella_happy_0_7-voice-20260125140835869-0d8f (缓存键: Bella_happy_0_7)
返回的音色名称: qwen-tts-vd-Bella_happy_0_7-voice-20260125140835869-0d8f

示例2: 再次使用相同参数（使用缓存）
✅ 使用已保存的音色: qwen-tts-vd-Bella_happy_0_7-voice-20260125140835869-0d8f (缓存键: Bella_happy_0_7)
返回的音色名称: qwen-tts-vd-Bella_happy_0_7-voice-20260125140835869-0d8f
是否相同: True

示例3: 查看所有已保存的音色
  Bella_happy_0_7 -> qwen-tts-vd-Bella_happy_0_7-voice-20260125140835869-0d8f

💡 提示：
1. 首次创建音色会调用API并产生费用（$0.2/次）
2. 后续使用相同参数会直接使用缓存，无需再次调用API
3. 音色缓存保存在: output/voice_cache.json
4. 重启notebook后，缓存仍然有效


In [72]:
# ==================== 重要提示 ====================

print("⚠️ 如果遇到 'TTSClient' object has no attribute 'voice_cache' 错误：")
print("\n解决方案：")
print("1. 重新运行 TTSClient 初始化 cell（包含 Voice Design 功能的版本）")
print("2. 运行: tts = TTSClient()")
print("3. 然后就可以使用音色缓存功能了")
print("\n" + "=" * 60)

# 检查当前实例
if hasattr(tts, 'voice_cache'):
    print("✅ 当前 TTSClient 实例已支持音色缓存功能")
    print(f"   已保存音色: {len(tts.voice_cache)} 个")
else:
    print("❌ 当前 TTSClient 实例不支持音色缓存功能")
    print("   请重新运行: tts = TTSClient()")
    print("\n💡 快速修复（临时方案）：")
    print("   tts.voice_cache = {}")
    print("   tts.voice_cache_file = PROJECT_ROOT / 'output' / 'voice_cache.json'")
    print("   然后重新运行演示 cell")

⚠️ 如果遇到 'TTSClient' object has no attribute 'voice_cache' 错误：

解决方案：
1. 重新运行 TTSClient 初始化 cell（包含 Voice Design 功能的版本）
2. 运行: tts = TTSClient()
3. 然后就可以使用音色缓存功能了

✅ 当前 TTSClient 实例已支持音色缓存功能
   已保存音色: 1 个


In [73]:
# 🎤 TTS 测试和音色切换工具
if tts.enabled:
    print("🎤 测试 TTS...")
    test_audio = tts.synthesize("你好，我是echuu AI主播，今天来给大家讲个有趣的故事！")
    if test_audio:
        print(f"✅ 生成音频: {len(test_audio)} bytes")
        display(Audio(test_audio, autoplay=False))
    
    # 显示 VTuber 推荐音色
    print(f"\n⭐ VTuber 推荐音色:")
    for voice in tts.VTUBER_RECOMMENDED:
        print(f"  • {voice}")
    
    # 显示所有可用音色（按分类）
    print(f"\n🎨 所有可用音色 ({len(tts.AVAILABLE_VOICES)}个):")
    categories = {
        "🎀 萌系女声": ["Cherry", "Serena", "Chelsie", "Momo", "Vivian", "Bunny", "Bella", "Mia", "Stella", "Nini"],
        "👩 知性女声": ["Katerina", "Maia", "Seren", "Bellona", "Elias", "Jennifer"],
        "👨 阳光男声": ["Ethan", "Ryan", "Aiden", "Moon", "Kai", "Nofish"],
        "👴 磁性男声": ["Andre", "Vincent", "Eldric_Sage", "Arthur", "Neil"],
        "🌍 方言特色": ["Li", "Marcus", "Roy", "Peter", "Sunny", "Eric"],
        "🌐 外语": ["Dolce", "Sohee", "Ono_Anna", "Lenn", "Emilien"],
        "👶 儿童/特殊": ["Mochi", "Pip", "Ebona"],
    }
    for cat, voices in categories.items():
        print(f"  {cat}: {', '.join(voices)}")
    
    # 测试情绪增强
    print("\n🔥 测试情绪增强 (emotion_boost=0.8):")
    emotional_audio = tts.synthesize("天哪！你们是不是不相信？！真的超级好笑！", emotion_boost=0.8)
    if emotional_audio:
        print(f"✅ 情绪化音频: {len(emotional_audio)} bytes")
        display(Audio(emotional_audio, autoplay=False))
else:
    print("⚠️ TTS 未启用，跳过测试")

# 💡 快速切换音色的辅助函数
def test_voice(voice_name: str, text: str = "你好，我是六螺，这是我的声音测试。"):
    """快速测试某个音色"""
    if not tts.enabled:
        print("⚠️ TTS 未启用")
        return
    
    tts.change_voice(voice_name)
    audio = tts.synthesize(text)
    if audio:
        print(f"🔊 播放音色: {voice_name}")
        display(Audio(audio, autoplay=True))

print("\n💡 使用示例:")
print("   test_voice('Chelsie', '大家好呀~ 我是千雪！今天来讲个有趣的故事~')  # 二次元女友")
print("   test_voice('Momo', '喵~ 今天也要开心哦！')  # 撒娇搞怪")
print("   test_voice('Bunny', '萌萌哒~ 宝贝们好呀！')  # 萝莉音")
print("   test_voice('Ethan', '各位好，今天来给大家讲个故事')  # 阳光男声")

🎤 测试 TTS...
✅ 生成音频: 21933 bytes



⭐ VTuber 推荐音色:
  • Chelsie
  • Momo
  • Bunny
  • Cherry
  • Vivian
  • Stella

🎨 所有可用音色 (41个):
  🎀 萌系女声: Cherry, Serena, Chelsie, Momo, Vivian, Bunny, Bella, Mia, Stella, Nini
  👩 知性女声: Katerina, Maia, Seren, Bellona, Elias, Jennifer
  👨 阳光男声: Ethan, Ryan, Aiden, Moon, Kai, Nofish
  👴 磁性男声: Andre, Vincent, Eldric_Sage, Arthur, Neil
  🌍 方言特色: Li, Marcus, Roy, Peter, Sunny, Eric
  🌐 外语: Dolce, Sohee, Ono_Anna, Lenn, Emilien
  👶 儿童/特殊: Mochi, Pip, Ebona

🔥 测试情绪增强 (emotion_boost=0.8):
✅ 情绪化音频: 16173 bytes



💡 使用示例:
   test_voice('Chelsie', '大家好呀~ 我是千雪！今天来讲个有趣的故事~')  # 二次元女友
   test_voice('Momo', '喵~ 今天也要开心哦！')  # 撒娇搞怪
   test_voice('Bunny', '萌萌哒~ 宝贝们好呀！')  # 萝莉音
   test_voice('Ethan', '各位好，今天来给大家讲个故事')  # 阳光男声


## Part 4: LLM 大模型客户端

In [74]:
class LLMClient:
    """Claude LLM 客户端"""
    
    def __init__(self):
        self.client = None
        self.model = os.getenv("claude-sonnet-4-20250514", "claude-sonnet-4-20250514")
        
        api_key = os.getenv("ANTHROPIC_API_KEY")
        if api_key:
            try:
                import anthropic
                self.client = anthropic.Anthropic(api_key=api_key)
                print(f"✅ LLM 已初始化: {self.model}")
            except ImportError:
                print("⚠️ anthropic 未安装")
        else:
            print("⚠️ 未设置 ANTHROPIC_API_KEY，使用 Mock 模式")
    
    def call(self, prompt: str, system: str = None, max_tokens: int = 1000) -> str:
        """调用 LLM"""
        if self.client:
            try:
                kwargs = {
                    "model": self.model,
                    "max_tokens": max_tokens,
                    "messages": [{"role": "user", "content": prompt}]
                }
                if system:
                    kwargs["system"] = system
                
                response = self.client.messages.create(**kwargs)
                return response.content[0].text
            except Exception as e:
                print(f"[LLM] 错误: {e}")
        
        return self._mock_response(prompt)
    
    def _mock_response(self, prompt: str) -> str:
        """Mock 响应"""
        import random
        
        if "JSON数组" in prompt or "script" in prompt.lower():
            return '''[
{"stage": "Hook", "goal": "用悬念开场", "attention": "audience", "speech_act": "elicit", "duration": 20, "cost": 0.3, "hint": "提问开场"},
{"stage": "Build-up", "goal": "铺垫背景", "attention": "self", "speech_act": "narrate", "duration": 40, "cost": 0.6, "hint": "描述情况"},
{"stage": "Climax", "goal": "关键转折", "attention": "self", "speech_act": "narrate", "duration": 40, "cost": 0.9, "hint": "情感爆发"},
{"stage": "Resolution", "goal": "总结感悟", "attention": "audience", "speech_act": "opine", "duration": 20, "cost": 0.4, "hint": "分享心得"}
]'''
        else:
            return json.dumps({
                "inner_monologue": random.choice(["让我想想...", "有意思...", "继续...", "嗯..."]),
                "decision": "continue",
                "speech": random.choice(["说起这个事儿啊...", "你们知道吗...", "我跟你们讲..."]),
                "emotion": "neutral"
            }, ensure_ascii=False)


# 初始化 LLM
llm = LLMClient()

✅ LLM 已初始化: claude-sonnet-4-20250514


## Part 5: 数据结构定义

包含：
- ScriptLine（剧本台词，含 V3 新字段）
- Danmaku（弹幕）
- PerformerMemory（记忆系统）
- PerformanceState（表演状态）

In [75]:
# ==================== 核心数据结构 ====================

@dataclass
class ScriptLine:
    """剧本台词（带key_info标注 + V3新增字段）"""
    id: str                          # 唯一ID
    text: str                        # 台词内容
    stage: str                       # Hook/Build-up/Climax/Resolution
    interruption_cost: float         # 0.0-1.0
    key_info: List[str] = field(default_factory=list)  # 🔑 关键信息点（用于匹配弹幕）
    
    # V3 新增字段
    disfluencies: List[str] = field(default_factory=list)  # 认知特征标记 ["数字模糊", "自我修正", "跑题"]
    emotion_break: Optional[Dict] = None  # 情绪断点 {"level": 1, "trigger": "想起画面", "recovery": "自嘲化解"}
    
    def __repr__(self):
        info_str = ", ".join(self.key_info[:2])
        emotion_str = f" [情绪L{self.emotion_break['level']}]" if self.emotion_break else ""
        return f"[{self.stage}]{emotion_str} {self.text[:30]}... (keys: {info_str})"


@dataclass
class Danmaku:
    """弹幕（带优先级计算）"""
    text: str
    user: str = "观众"
    is_sc: bool = False
    amount: int = 0              # SC金额
    
    # 计算得出的属性
    relevance: float = 0.0       # 与当前话题的相关性 0.0-1.0
    priority: float = 0.0        # 最终优先级
    
    @classmethod
    def from_text(cls, text: str) -> "Danmaku":
        """解析弹幕"""
        is_sc = False
        amount = 0
        
        # 检测SC
        if "SC" in text or "¥" in text or "$" in text:
            is_sc = True
            # 简单提取金额
            import re
            match = re.search(r'[¥$]?\s*(\d+)', text)
            if match:
                amount = int(match.group(1))
        
        return cls(text=text, is_sc=is_sc, amount=amount)
    
    def is_question(self) -> bool:
        """判断是否是问题"""
        return "?" in self.text or "？" in self.text


@dataclass
class PerformerMemory:
    """
    记忆系统 - 可视化展示AI记住了什么
    """
    
    # 📖 剧本进度
    script_progress: Dict = field(default_factory=lambda: {
        "current_line": 0,
        "total_lines": 0,
        "completed_stages": [],
        "current_stage": "Hook",
    })
    
    # 💬 弹幕记忆
    danmaku_memory: Dict = field(default_factory=lambda: {
        "received": [],          # 收到的所有弹幕
        "responded": [],         # 已回应的
        "ignored": [],           # 被忽略的
        "pending_questions": [], # 待回答的问题
    })
    
    # 🤝 承诺追踪
    promises: List[Dict] = field(default_factory=list)
    # 例如: {"content": "室友生不生气", "made_at_step": 3, "fulfilled": False, "answer_at_line": 9}
    
    # 🎭 故事要点
    story_points: Dict = field(default_factory=lambda: {
        "mentioned": [],         # 已经提到的信息
        "upcoming": [],          # 即将要说的关键点
        "revealed": [],          # 已揭示的"答案"
    })
    
    # 😊 情绪轨迹
    emotion_track: List[Dict] = field(default_factory=list)
    
    def to_display(self) -> str:
        """生成用户可见的记忆状态"""
        lines = []
        lines.append("┌─────────────────────────────────────────────┐")
        lines.append("│ 🧠 AI记忆状态                               │")
        lines.append("├─────────────────────────────────────────────┤")
        
        # 剧本进度
        prog = self.script_progress
        current = prog.get("current_line", 0)
        total = prog.get("total_lines", 0)
        if total > 0:
            percent = int(current / total * 10)
            bar = "█" * percent + "░" * (10 - percent)
            lines.append(f"│ 📖 剧本: [{bar}] {current}/{total} ({prog.get('current_stage', '?')})  │")
        
        # 弹幕记忆
        dm = self.danmaku_memory
        responded = len(dm.get("responded", []))
        pending = len(dm.get("pending_questions", []))
        lines.append(f"│ 💬 弹幕: 已回应{responded}条, 待回答{pending}个问题         │")
        
        # 承诺
        unfulfilled = [p for p in self.promises if not p.get("fulfilled", False)]
        if unfulfilled:
            lines.append("│ 🤝 待兑现承诺:                              │")
            for p in unfulfilled[:2]:
                content = p.get("content", "")[:20]
                lines.append(f"│    • {content}...                         │")
        
        # V3: 情绪轨迹
        if self.emotion_track:
            lines.append("│ 🎭 情绪轨迹:                                │")
            for emo in self.emotion_track[-2:]:
                level_name = {1: "微破防", 2: "明显破防", 3: "完全破防"}.get(emo.get('level', 0), "?")
                trigger = emo.get('trigger', '')[:12]
                lines.append(f"│    {level_name}: {trigger}...              │")
        
        # 已提到要点
        mentioned = self.story_points.get("mentioned", [])
        if mentioned:
            lines.append("│ ✅ 已提到:                                  │")
            for point in mentioned[-3:]:
                lines.append(f"│    ✓ {point[:20]}...                      │")
        
        lines.append("└─────────────────────────────────────────────┘")
        return "\n".join(lines)
    
    def to_context(self) -> str:
        """生成给LLM的上下文摘要"""
        parts = []
        
        # 剧本进度
        prog = self.script_progress
        parts.append(f"剧本进度: {prog.get('current_line')}/{prog.get('total_lines')} ({prog.get('current_stage')})")
        
        # 已提到的信息
        mentioned = self.story_points.get("mentioned", [])
        if mentioned:
            parts.append(f"已提到: {', '.join(mentioned[-5:])}")
        
        # 待回答问题
        pending = self.danmaku_memory.get("pending_questions", [])
        if pending:
            parts.append(f"待回答: {', '.join(q[:20] for q in pending[:3])}")
        
        # 未兑现承诺
        unfulfilled = [p for p in self.promises if not p.get("fulfilled", False)]
        if unfulfilled:
            parts.append(f"待兑现承诺: {', '.join(p['content'][:15] for p in unfulfilled[:2])}")
        
        return " | ".join(parts)


@dataclass
class PerformanceState:
    """表演状态"""
    name: str
    persona: str
    background: str
    topic: str
    
    # 完整剧本（预生成）
    script_lines: List[ScriptLine] = field(default_factory=list)
    current_line_idx: int = 0
    current_step: int = 0
    
    # 记忆系统
    memory: PerformerMemory = field(default_factory=PerformerMemory)
    
    # 弹幕队列
    danmaku_queue: List[Danmaku] = field(default_factory=list)
    
    # 口癖
    catchphrases: List[str] = field(default_factory=list)

## Part 6: 弹幕处理系统

包含：
- DanmakuEvaluator（优先级评估）
- DanmakuHandler（统一处理器）
- SC 响应优化

In [76]:
# ==================== 工具函数 ====================

def extract_keywords(text: str) -> List[str]:
    """提取关键词（简单版）"""
    import re
    # 移除标点
    text = re.sub(r'[，。！？、：；""''（）【】《》…~]', ' ', text)
    # 分词（简单按空格）
    words = text.split()
    # 过滤停用词
    stopwords = {'的', '了', '在', '是', '我', '你', '他', '她', '它', '们', 
                 '这', '那', '有', '和', '就', '不', '也', '都', '说', '很',
                 '吗', '吧', '呢', '啊', '哦', '嗯', '哈', '呀'}
    keywords = [w for w in words if len(w) >= 2 and w not in stopwords]
    return keywords[:5]  # 最多5个


# ==================== 弹幕处理器 ====================

class DanmakuEvaluator:
    """
    弹幕评估器 - 计算优先级
    
    优先级公式:
    priority = base_score + relevance_bonus + sc_bonus
    """
    
    def evaluate(self, danmaku: Danmaku, state: PerformanceState) -> Danmaku:
        """评估单条弹幕"""
        
        # 1. 基础分
        if danmaku.is_question():
            base = 0.3  # 问题类
        elif any(kw in danmaku.text for kw in ["哈哈", "笑死", "真的假的", "！", "牛"]):
            base = 0.15  # 情绪反馈类
        else:
            base = 0.1  # 普通评论
        
        # 2. 相关性评估
        relevance = self._calc_relevance(danmaku.text, state)
        danmaku.relevance = relevance
        
        if relevance > 0.7:
            relevance_bonus = 0.4  # 高相关
        elif relevance > 0.4:
            relevance_bonus = 0.2  # 中相关
        else:
            relevance_bonus = 0.0  # 低相关
        
        # 3. SC加成
        if danmaku.is_sc:
            if danmaku.amount >= 200:
                sc_bonus = 0.7
            elif danmaku.amount >= 100:
                sc_bonus = 0.5
            elif danmaku.amount >= 50:
                sc_bonus = 0.3
            else:
                sc_bonus = 0.2
        else:
            sc_bonus = 0.0
        
        danmaku.priority = base + relevance_bonus + sc_bonus
        
        return danmaku
    
    def _calc_relevance(self, text: str, state: PerformanceState) -> float:
        """计算弹幕与当前上下文的相关性"""
        # 提取当前故事的关键词
        story_keywords = set()
        for info in state.memory.story_points.get("mentioned", []):
            story_keywords.update(extract_keywords(info))
        for info in state.memory.story_points.get("upcoming", []):
            story_keywords.update(extract_keywords(info))
        
        # 如果没有故事关键词，检查话题
        if not story_keywords:
            story_keywords.update(extract_keywords(state.topic))
        
        # 提取弹幕关键词
        danmaku_keywords = set(extract_keywords(text))
        
        # 计算重叠度
        if not story_keywords:
            return 0.0
        overlap = len(danmaku_keywords & story_keywords)
        return min(overlap / 2, 1.0)  # 2个关键词重叠就算高相关


class DanmakuHandler:
    """
    统一弹幕处理器
    
    所有弹幕走同一流程，根据优先级决定：
    1. 是否打断
    2. 是否复读
    3. 如何回应
    4. 是否改变叙事
    """
    
    def __init__(self, evaluator: DanmakuEvaluator):
        self.evaluator = evaluator
    
    def handle(self, danmaku: Danmaku, state: PerformanceState) -> Dict:
        """
        处理弹幕
        
        Returns:
            {
                "should_interrupt": bool,
                "echo": str,
                "response": str,
                "action": str,  # continue/jump/tease/improvise
            }
        """
        
        # 评估优先级
        danmaku = self.evaluator.evaluate(danmaku, state)
        
        # 获取当前cost
        if state.current_line_idx >= len(state.script_lines):
            current_cost = 0.2  # 结尾阶段
        else:
            current_line = state.script_lines[state.current_line_idx]
            current_cost = current_line.interruption_cost
        
        # 决策：是否打断
        should_interrupt = danmaku.priority > current_cost
        
        if not should_interrupt:
            return {
                "should_interrupt": False,
                "action": "ignore",
                "priority": danmaku.priority,
                "cost": current_cost
            }
        
        # 决定是否复读
        echo = self._maybe_echo(danmaku)
        
        # 查找答案位置
        answer_loc = self._find_answer(danmaku.text, state)
        
        # 生成回应策略
        action = self._decide_action(danmaku, answer_loc, state)
        
        return {
            "should_interrupt": True,
            "echo": echo,
            "action": action,
            "answer_loc": answer_loc,
            "priority": danmaku.priority,
            "cost": current_cost,
            "relevance": danmaku.relevance
        }
    
    def _maybe_echo(self, danmaku: Danmaku) -> str:
        """决定是否复读弹幕"""
        if danmaku.is_sc:
            return f"诶有SC！有人说：{danmaku.text}"
        elif danmaku.priority > 0.5:
            return f"有人说{danmaku.text}，"
        elif danmaku.is_question():
            return f"有人问{danmaku.text}，"
        else:
            return ""  # 不复读
    
    def _find_answer(self, question: str, state: PerformanceState) -> Dict:
        """在剧本中查找问题的答案位置"""
        keywords = extract_keywords(question)
        current_idx = state.current_line_idx
        
        for i, line in enumerate(state.script_lines):
            if i < current_idx:
                continue  # 跳过已经说过的
            
            for info in line.key_info:
                if any(kw in info for kw in keywords):
                    return {
                        "found": True,
                        "line_idx": i,
                        "distance": i - current_idx,
                        "answer_hint": info,
                    }
        
        return {"found": False}
    
    def _decide_action(self, danmaku: Danmaku, answer_loc: Dict, 
                       state: PerformanceState) -> str:
        """
        决定叙事动作
        
        - continue: 回应后继续当前剧本
        - jump: 跳跃到答案位置
        - reveal: 直接揭晓答案
        - tease: 吊胃口
        - improvise: 即兴回答
        """
        
        if not answer_loc.get("found", False):
            return "improvise"
        
        distance = answer_loc.get("distance", 0)
        
        # 根据优先级和距离决定策略
        if danmaku.priority > 0.8:
            # 高优先级（大额SC或高相关问题）
            if distance <= 3:
                return "jump"
            else:
                return "tease"
        elif danmaku.priority > 0.5:
            # 中优先级
            if distance <= 2:
                return "continue"
            else:
                return "tease"
        else:
            # 低优先级
            return "continue"


# 初始化
evaluator = DanmakuEvaluator()
danmaku_handler = DanmakuHandler(evaluator)

print("✅ 弹幕处理器已初始化")
print("  • 优先级公式: priority = base + relevance_bonus + sc_bonus")
print("  • 相关性是最重要因素")


✅ 弹幕处理器已初始化
  • 优先级公式: priority = base + relevance_bonus + sc_bonus
  • 相关性是最重要因素


In [77]:
# 🔥 SC响应优化：提高SC优先级，确保主播积极响应

# 重写 DanmakuEvaluator.evaluate 方法，提高SC加成
original_evaluate = DanmakuEvaluator.evaluate

def enhanced_evaluate(self, danmaku: Danmaku, state: PerformanceState) -> Danmaku:
    """增强版评估：大幅提高SC优先级"""
    # 调用原始评估
    danmaku = original_evaluate(self, danmaku, state)
    
    # 🔥 SC特权增强
    if danmaku.is_sc:
        # 重新计算SC加成（更高的值）
        if danmaku.amount >= 200:
            sc_bonus_extra = 0.3  # 从0.7提升到1.0
        elif danmaku.amount >= 100:
            sc_bonus_extra = 0.3  # 从0.5提升到0.8
        elif danmaku.amount >= 50:
            sc_bonus_extra = 0.3  # 从0.3提升到0.6
        else:
            sc_bonus_extra = 0.2  # 从0.2提升到0.4
        
        danmaku.priority += sc_bonus_extra
        
        # SC特权：低相关性也给保底bonus
        if danmaku.relevance < 0.3:
            danmaku.priority += 0.2  # 保底优先级
    
    return danmaku

# 应用增强
DanmakuEvaluator.evaluate = enhanced_evaluate

# 重新创建弹幕处理器
evaluator = DanmakuEvaluator()
danmaku_handler = DanmakuHandler(evaluator)

print("✅ SC响应已优化：")
print("   • ¥50 SC优先级提升至 ~1.1（可打断大部分阶段）")
print("   • ¥100+ SC优先级提升至 1.3+（可打断任何阶段）")
print("   • 低相关性SC也会获得保底优先级")
print("   • improvise模式已改为积极响应，不再说'与话题无关'")


✅ SC响应已优化：
   • ¥50 SC优先级提升至 ~1.1（可打断大部分阶段）
   • ¥100+ SC优先级提升至 1.3+（可打断任何阶段）
   • 低相关性SC也会获得保底优先级
   • improvise模式已改为积极响应，不再说'与话题无关'


## Part 7: 剧本生成器

包含：
- ExampleSampler（Few-shot 随机采样器）
- **ScriptGeneratorV3**（主力生成器，默认使用）
- ScriptGeneratorV2（旧版，保留兼容）

### 7.1 ExampleSampler - Few-shot 随机采样器

从 30 个真实 VTuber 片段中随机采样，确保：
1. 至少 1 个高情绪爆发的片段
2. 至少 1 个有跑题/自我修正的片段
3. 随机性保证多样性

In [78]:
# ==================== ExampleSampler: Few-shot 随机采样器 ====================

import random
import json
from pathlib import Path

class ExampleSampler:
    """
    从 30 个真实 VTuber 片段中随机采样 few-shot examples
    
    特点：
    1. 按特征分类（高情绪 vs 日常 vs 叙事）
    2. sample_diverse(n=3) 确保采样多样性
    3. 优先选有自我修正、情绪词的片段
    """
    
    def __init__(self, clips_path: str):
        """
        加载所有 clips 数据
        
        Args:
            clips_path: vtuber_raw_clips_for_notebook_full_30_cleaned.jsonl 的路径
        """
        self.clips = []
        clips_file = Path(clips_path)
        
        if clips_file.exists():
            with open(clips_file, 'r', encoding='utf-8') as f:
                for line in f:
                    if line.strip():
                        self.clips.append(json.loads(line))
            print(f"📚 ExampleSampler: 加载了 {len(self.clips)} 个 clips")
        else:
            print(f"⚠️ 警告: clips 文件不存在: {clips_path}")
        
        # 按特征分类
        self.categorize_clips()
    
    def categorize_clips(self):
        """将 clips 按特征分类"""
        self.by_emotion = {
            'high_emotion': [],  # 有明显情绪爆发的
            'casual': [],        # 日常闲聊的
            'storytelling': []   # 讲故事的
        }
        
        self.by_structure = {
            'linear': [],        # 线性叙事
            'digressive': [],    # 跑题多的
            'interactive': []    # 弹幕互动多的
        }
        
        # 中文高情绪关键词
        zh_high_emotion = ['爆发', '紧张', '释然', '感动', '破防', '愤怒', '震惊', '激动']
        zh_casual = ['轻松', '调侃', '自嘲', '搞笑', '宠溺']
        
        # 英文高情绪关键词
        en_high_emotion = ['angry', 'rage', 'crying', 'emotional', 'sad', 'vulnerable', 'break']
        en_casual = ['funny', 'chill', 'relax', 'joking', 'sarcastic']
        
        for clip in self.clips:
            notes = clip.get('notes', {})
            lang = clip.get('language', 'zh')
            
            # 根据 emotion 字段分类
            emotion = notes.get('emotion', '').lower()
            feature = notes.get('feature', '').lower()
            
            # 高情绪检测
            high_markers = zh_high_emotion if lang == 'zh' else en_high_emotion
            casual_markers = zh_casual if lang == 'zh' else en_casual
            
            is_high = any(word in emotion or word in feature for word in high_markers)
            is_casual = any(word in emotion or word in feature for word in casual_markers)
            
            # 有 ** 标记的是经典案例，视为高情绪
            if '**' in feature:
                is_high = True
            
            if is_high:
                self.by_emotion['high_emotion'].append(clip)
            elif is_casual:
                self.by_emotion['casual'].append(clip)
            else:
                self.by_emotion['storytelling'].append(clip)
            
            # 根据 structure 字段分类
            structure = notes.get('structure', '')
            if '跑题' in structure or '插入' in structure or 'digress' in structure.lower():
                self.by_structure['digressive'].append(clip)
            elif '互动' in structure or '弹幕' in structure or 'interactive' in structure.lower():
                self.by_structure['interactive'].append(clip)
            else:
                self.by_structure['linear'].append(clip)
        
        print(f"   情绪分类: 高情绪={len(self.by_emotion['high_emotion'])}, "
              f"日常={len(self.by_emotion['casual'])}, "
              f"叙事={len(self.by_emotion['storytelling'])}")
        print(f"   结构分类: 跑题={len(self.by_structure['digressive'])}, "
              f"互动={len(self.by_structure['interactive'])}, "
              f"线性={len(self.by_structure['linear'])}")
    
    def sample_diverse(self, n: int = 3, language: str = None) -> List[Dict]:
        """
        采样 n 个多样化的 examples
        
        确保：
        1. 至少 1 个有情绪爆发
        2. 至少 1 个有跑题/自我修正
        3. 随机性
        
        Args:
            n: 采样数量
            language: 语言过滤 ('zh' 或 'en')，None 表示不过滤
        """
        # 语言过滤
        if language:
            available = [c for c in self.clips if c.get('language') == language]
            high_emotion = [c for c in self.by_emotion['high_emotion'] if c.get('language') == language]
            digressive = [c for c in self.by_structure['digressive'] if c.get('language') == language]
        else:
            available = self.clips
            high_emotion = self.by_emotion['high_emotion']
            digressive = self.by_structure['digressive']
        
        if not available:
            print(f"⚠️ 没有可用的 clips (language={language})")
            return []
        
        samples = []
        
        # 必选：1 个高情绪的（如果有的话）
        if high_emotion:
            samples.append(random.choice(high_emotion))
        
        # 必选：1 个跑题多的（如果有的话）
        digressive_available = [c for c in digressive if c not in samples]
        if digressive_available:
            samples.append(random.choice(digressive_available))
        
        # 剩余随机填充
        remaining = [c for c in available if c not in samples]
        while len(samples) < n and remaining:
            choice = random.choice(remaining)
            samples.append(choice)
            remaining.remove(choice)
        
        # 打乱顺序，避免固定模式
        random.shuffle(samples)
        return samples
    
    def extract_transcript_segments(self, clip: Dict, max_segments: int = 3) -> str:
        """
        从一个 clip 中提取有代表性的 transcript 片段
        
        优先选择：有自我修正、有情绪词、有跑题的片段
        """
        transcript = clip.get('transcript', [])
        if not transcript:
            return ""
        
        lang = clip.get('language', 'zh')
        
        # 中文特征词
        zh_self_correct = ['不对', '还是', '来着', '我的意思是', '不是那个', '应该是', '好像是']
        zh_emotion = ['救命', '天哪', '我的天', '哎呀', '卧槽', '太', '真的', '可怕']
        zh_digress = ['对了', '说起这个', '诶', '话说', '等等', '不是']
        
        # 英文特征词
        en_self_correct = ["wait", "no", "actually", "I mean", "not that", "well"]
        en_emotion = ["oh my god", "holy", "what the", "damn", "crazy", "insane"]
        en_digress = ["anyway", "by the way", "speaking of", "oh", "wait"]
        
        self_correct = zh_self_correct if lang == 'zh' else en_self_correct
        emotion_words = zh_emotion if lang == 'zh' else en_emotion
        digress_words = zh_digress if lang == 'zh' else en_digress
        
        # 评分每个 segment
        scored_segments = []
        for seg in transcript:
            text = seg.get('text', '').lower()
            score = 0
            
            # 有自我修正特征加分
            if any(marker.lower() in text for marker in self_correct):
                score += 3
            
            # 有情绪词加分
            if any(marker.lower() in text for marker in emotion_words):
                score += 2
            
            # 有跑题特征加分
            if any(marker.lower() in text for marker in digress_words):
                score += 2
            
            # 有重复词语加分（中文）
            if lang == 'zh' and '，' in seg.get('text', ''):
                parts = seg.get('text', '').split('，')
                for i in range(len(parts)-1):
                    if len(parts[i]) >= 2 and len(parts[i+1]) >= 2:
                        if parts[i][-2:] == parts[i+1][:2]:
                            score += 1
            
            # 长度适中加分
            text_len = len(seg.get('text', ''))
            if 50 < text_len < 300:
                score += 1
            
            scored_segments.append((score, seg))
        
        # 取分数最高的几个
        scored_segments.sort(key=lambda x: x[0], reverse=True)
        selected = scored_segments[:max_segments]
        
        # 按时间顺序重排
        selected.sort(key=lambda x: x[1].get('t', 0) or 0)
        
        result = []
        for _, seg in selected:
            result.append(seg.get('text', ''))
        
        return '\n'.join(result)
    
    def format_as_fewshot(self, clips: List[Dict]) -> str:
        """将采样的 clips 格式化为 few-shot prompt"""
        output = []
        
        for i, clip in enumerate(clips, 1):
            title = clip.get('title', '未知')
            notes = clip.get('notes', {})
            lang = clip.get('language', 'zh')
            
            # 提取关键特征
            features = []
            if notes.get('habit'):
                features.append(f"口癖: {notes['habit']}")
            if notes.get('emotion'):
                features.append(f"情绪: {notes['emotion']}")
            if notes.get('feature'):
                # 移除 ** 标记
                feat = notes['feature'].replace('**', '')
                features.append(f"特点: {feat}")
            
            # 提取代表性片段
            segments = self.extract_transcript_segments(clip)
            
            lang_label = "中文" if lang == "zh" else "英文"
            
            output.append(f"""
### 真实案例 {i}: {title} ({lang_label})
{' | '.join(features)}

**原文片段（注意口语特征）：**
```
{segments}
```
""")
        
        return '\n'.join(output)
    
    def get_random_examples(self, n: int = 3, language: str = 'zh') -> str:
        """
        一键获取格式化的 few-shot examples
        
        Args:
            n: 采样数量
            language: 语言 ('zh' 或 'en')
        
        Returns:
            格式化的 few-shot prompt 字符串
        """
        samples = self.sample_diverse(n=n, language=language)
        if not samples:
            return "（无可用的 few-shot examples）"
        return self.format_as_fewshot(samples)


# ==================== 初始化 ExampleSampler ====================
CLIPS_DATA_PATH = PROJECT_ROOT / "data" / "vtuber_raw_clips_for_notebook_full_30_cleaned.jsonl"

try:
    example_sampler = ExampleSampler(str(CLIPS_DATA_PATH))
    print("✅ ExampleSampler 已初始化")
except Exception as e:
    example_sampler = None
    print(f"⚠️ ExampleSampler 初始化失败: {e}")

📚 ExampleSampler: 加载了 30 个 clips
   情绪分类: 高情绪=19, 日常=5, 叙事=6
   结构分类: 跑题=0, 互动=6, 线性=24
✅ ExampleSampler 已初始化


### 7.2 ScriptGeneratorV2（旧版 - 保留兼容）

> ⚠️ V2 已不是默认生成器，仅保留用于向后兼容。**新项目请使用下方的 V3。**

In [ ]:
class ScriptGeneratorV2:
    """
    剧本生成器 V2 - 预生成完整台词（带 key_info 标注）
    """
    
    SYSTEM_PROMPT = """你是资深VTuber脚本架构师（真实感专项）。

## 目标
生成极具"亲历者感"的直播脚本，让观众觉得这是主播昨晚刚发生的真实糗事，而不是AI编造的故事。

⚠️ **核心要求：每个叙事单元必须是200-400字的完整段落**

## 🔥 4个核心生成算法（必须遵守）

### 1️⃣ 【模糊记忆算法】
人类回忆时，数字和时间是不精确的。**强制要求加入"自我修正"**。

❌ 错误："我等了30分钟。"
✅ 正确："我等了大概半小时吧，不对，可能也就二十分钟，因为我刷了会儿抖音就排到了。"

❌ 错误："我买了一瓶可乐。"
✅ 正确："一个月给自己订的，一个月喝三瓶可乐，三瓶还是四瓶来着。"

### 2️⃣ 【物理逻辑锚点】
寻找场景下最琐碎、最"没用"、甚至有点"恶心/尴尬"的物理细节。

❌ 错误："猫很可爱"
✅ 正确："它在猫砂盆里刨坑的声音像在装修，而且那股便宜猫砂的工业桃子味特别冲。"

❌ 错误："室友有很多零食"
✅ 正确："冰箱里头，我记得好清楚是海苔炒腰果，搞了一大盒，放了那，放了那。"

### 3️⃣ 【心理滑坡路径】
不要直接展示结果，要展示"心怀侥幸 → 尝试边缘试探 → 意外发生 → 徒劳补救"的全过程。

❌ 错误："我偷吃了腰果。"
✅ 正确："我想着我吃一个腰果，他回来他又不数，他应该不知道吧，偷吃一个。第二天又想着，要不我再偷吃一个吧，他回来应该不知道吧。结果我吃着吃着，他还没回，我已经把里面的腰果吃完了。"

### 4️⃣ 【元评论介入】
在叙事中穿插对当下的评价，与观众对话。

- "救命，我现在想起来手心还在冒汗。"
- "弹幕别笑了，换你你也懵。"
- "你们能理解那种感觉吗？"
- "我跟你们说，真的..."

---

## ❌ 绝对禁止（这些都太短了！）
```
❌ "那时候我很穷" 
❌ "室友是富二代"
❌ "我偷吃了腰果"
❌ "后来坦白了，室友很宽容"
```
↑ **短句总结，缺乏细节！绝对不要！**

### ✅ 正确示范（应用4个算法的长段落）

**示例1: 铺垫穷困（约280字）**
✓ 模糊记忆："三瓶还是四瓶来着"
✓ 物理细节："酱油拌饭"、"雪碧太贵"
✓ 元评论："你们知道吗"

"我那个时候一个月给自己订的，一个月喝三瓶可乐，三瓶还是四瓶来着，因为我特别爱喝可乐，但是当时没钱。你们知道我那个时候一天吃多少钱吗？500日元，折合人民币20多块钱。早上100，中午200，晚上200。早上一个饭团，中午一碗面或者一碗饭加一点配菜，晚上也是这样。我当时穷到什么程度呢，就是酱油拌饭，就很奢侈了。我那时候可不是雪碧拌饭，雪碧太贵了。救命，我现在想起来都觉得，我那个时候怎么活下来的？"

**示例2: 心理滑坡（约320字）**
✓ 心理滑坡：侥幸→试探→失控
✓ 物理细节："海苔盖回去"
✓ 元评论："我跟你说"

"我看着那个冰箱里的，他的那个海苔炒腰果，哎呀我好馋呀。我当时就想着，我吃一个腰果，他回来他又不数，他应该不知道吧。我就偷偷打开那个盖子，拿了一个，就一个，我跟你说我真的就拿了一个。我把它吃掉，然后把盖子盖好，把那个海苔盖回去。第二天我又想着，要不我再偷吃一个吧，反正他也不数的，对吧。你们能理解那种感觉吗？就是明知道不对，但就是控制不住。结果我吃着吃着，他还没回来呢，我发现我已经把里面的腰果吃完了。只剩下海苔了。弹幕别笑，换你你也会这样的！"

**示例3: 情绪爆发+元评论（约340字）**
✓ 物理细节："在里头一翻腾"、"盖子"
✓ 徒劳补救："想用海苔掩藏"
✓ 元评论："你们懂吗"、"真的"

"把我紧张的都睡不着觉了，每天晚上我都没睡着，因为当我意识到，我在里头一翻腾，发现腰果没了。哎呀我心想这可怎么办。你们懂吗，那种心理负担真的可重了。第二天我就实在是，我太惦记这事了，我压力太大了，我就给他坦白了。我说我有一个事情，我想跟你坦白，你不要生气。他说啥呀？我说，嗯，我把你那个腰果吃掉了，我一开始想给你留一点的，但是它太好吃了，我一天吃两个把它吃完了。海苔我想吃，但那个海苔占的体积比较大嘛，就是我本来是想用海苔掩藏那个，盖住那个腰果的。救命，我现在说出来都觉得自己当时的想法好蠢。"
```

## 写作技巧清单（每个单元必备）

### ✅ 4个算法的具体应用

**1. 模糊记忆算法（必须！）**
- 数字要修正："三瓶还是四瓶来着"、"大概半小时吧，不对，可能二十分钟"
- 时间要犹豫："那天是周二还是周三来着"
- 记忆要不确定："我记得好像是"、"应该是"

**2. 物理逻辑锚点（必须！）**
- 找"没用"的细节：猫砂的气味、OA按钮的颜色、饭团的温度
- 找"尴尬"的细节：手心出汗、肚子咕咕叫、袜子没穿对
- 用品牌和型号：不要说"手机"，说"iPhone 13"；不要说"零食"，说"海苔炒腰果"

**3. 心理滑坡路径（必须！）**
- 第一步：心怀侥幸 - "就一个，他应该不会发现"
- 第二步：边缘试探 - "要不再来一个？"
- 第三步：意外发生 - "诶怎么全没了？"
- 第四步：徒劳补救 - "我用海苔盖一盖"

**4. 元评论介入（必须！）**
- 对当下评价："救命"、"我现在想起来都..."
- 与观众对话："你们懂吗"、"弹幕别笑"、"换你你也..."
- 自我吐槽："我当时怎么这么蠢"

### ✅ 其他必备元素
- **完整对话** - 写出来，不要转述："他说'你想吃就吃呀'。我说真的吗？"
- **时间推进** - "第二天"、"结果"、"后来"
- **口癖** - 根据角色设定穿插

### ✅ 叙事节奏
- 铺垫要充分：先说背景→发现诱惑→起贪念→第一次→第二次...
- 高潮要有张力：侥幸→失控→慌张→坦白→意外结局
- 用内心戏和物理细节拉长时间感

## 叙事结构（8-12单元，总计240秒）
- Hook (1-2单元): 开场悬念，每单元200-300字 - cost 0.2-0.4
- Build-up (3-5单元): 详细铺垫，每单元250-400字 - cost 0.5-0.7
- Climax (2-3单元): 情绪爆发，每单元200-350字 - cost 0.8-0.95
- Resolution (1-2单元): 总结升华，每单元150-250字 - cost 0.3-0.5

## key_info 标注（用于匹配弹幕问题）

"我那个时候一个月给自己订的，一个月喝三瓶可乐"
key_info: ["留学穷困", "可乐限量", "节约生活"]

"我想我吃一个腰果，他回来他又不数，他应该不知道吧"
key_info: ["第一次偷吃", "侥幸心理", "内心挣扎"]

"他说你想吃就吃呀随便吃呀"
key_info: ["室友反应", "室友宽容", "没生气", "危机解除"]  ← 这是"室友生气吗"的答案！"""
    
    def __init__(self, llm: LLMClient, analyzer: PatternAnalyzer = None):
        self.llm = llm
        self.analyzer = analyzer
    
    def generate(self, name: str, persona: str, background: str, 
                 topic: str, language: str = "zh") -> List[ScriptLine]:
        """生成完整剧本"""
        
        # 获取参考数据
        catchphrases = ""
        example_hooks = ""
        
        if self.analyzer:
            cps = self.analyzer.extract_catchphrases(language)[:5]
            catchphrases = ", ".join(f'"{c}"' for c, _ in cps)
            
            hooks = self.analyzer.extract_hooks(language)[:2]
            example_hooks = "\n".join(f'- "{h[:50]}..."' for h in hooks)
        
        prompt = f"""为 {name} 生成关于"{topic}"的完整直播剧本。

## 角色信息
- 名字: {name}
- 人设: {persona}
- 背景: {background}
- 话题: {topic}
- 口癖: {catchphrases or "自然即可"}

## 开场参考
{example_hooks or "（无）"}

## ⚠️ 核心要求（严格遵守！）

**目标时长: 240秒 (4分钟)**
**需要生成: 8-12个叙事单元**
**每个单元: 200-400字**
**总字数: 2000-4000字**

## 结构分配
- **Hook（开场）**: 1-2单元，每单元200-300字 - 悬念/问题开场
- **Build-up（铺垫）**: 3-5单元，每单元250-400字 - 详细背景、生活细节、内心戏
- **Climax（高潮）**: 2-3单元，每单元200-350字 - 转折、情绪爆发、揭晓答案
- **Resolution（收尾）**: 1-2单元，每单元150-250字 - 情绪释放、总结升华

⚠️ **每个单元必须200字以上！短于200字说明细节不够！**

## ⚠️ 核心要求：必须应用4个算法！

**每个单元必须包含：**

1. **【模糊记忆】** - 数字/时间要不确定、要自我修正
   - "三瓶还是四瓶来着"、"大概半小时吧不对可能二十分钟"

2. **【物理锚点】** - 琐碎的、尴尬的、"没用"的物理细节
   - "猫砂的工业桃子味"、"OA按钮的屎黄色"、"手心全是汗"

3. **【心理滑坡】** - 侥幸→试探→失控→补救的完整过程
   - 不要直接说结果，要展示心路历程

4. **【元评论】** - 与观众对话、对当下评价
   - "救命"、"你们懂吗"、"弹幕别笑"、"我现在想起来都..."

**其他必备：**
- 完整对话（不要转述）
- 时间推进（"第二天"、"结果"）
- 角色口癖

### 不要写总结式（❌）
❌ "那时候我很穷" 
✅ "我那个时候一个月给自己订的，一个月喝三瓶可乐，三瓶还是四瓶，因为我特别爱喝，但是当时没钱"

## 输出格式

JSON数组：
[
  {{
    "text": "这里是200-400字的长段落，有口癖、有细节、有内心戏、有完整对话...",
    "stage": "Hook",
    "cost": 0.3,
    "key_info": ["关键信息1", "关键信息2"]
  }},
  ...8-12个单元...
]

要求：
- **必须生成8-12个单元，每单元200-400字**
- 每单元推进新信息，不重复
- key_info准确（用于匹配弹幕）
- 符合{name}人设和口癖
- 口语化，有停顿，有情绪

⚠️ **质量检查标准：**
- 每个单元必须200字以上
- 每个单元必须应用4个算法中的至少3个
- 读起来像真实的回忆，不像AI生成的故事
- 有"糗事"的质感：尴尬、无奈、好笑

## 🎯 终极目标
让观众看完后的反应是："哈哈哈我也有过！" 而不是 "这故事编得不错"。

只输出JSON数组。"""
        
        response = self.llm.call(prompt, system=self.SYSTEM_PROMPT, max_tokens=4000)
        
        try:
            # 解析JSON
            if "```" in response:
                json_str = response.split("```")[1]
                if json_str.startswith("json"):
                    json_str = json_str[4:]
            else:
                json_str = response
            
            lines_data = json.loads(json_str.strip())
            
            # 转换为 ScriptLine
            script_lines = []
            for i, line_data in enumerate(lines_data):
                script_lines.append(ScriptLine(
                    id=f"line_{i}",
                    text=line_data.get("text", ""),
                    stage=line_data.get("stage", "Unknown"),
                    interruption_cost=line_data.get("cost", 0.5),
                    key_info=line_data.get("key_info", [])
                ))
            
            print(f"✅ 生成了 {len(script_lines)} 句台词")
            return script_lines
            
        except Exception as e:
            print(f"[Script] 解析失败: {e}")
            print(f"响应内容: {response[:200]}...")
            return self._fallback_script(name, topic)
    
    def _fallback_script(self, name: str, topic: str) -> List[ScriptLine]:
        """后备剧本（长段落版本）"""
        return [
            # Hook (1-2单元)
            ScriptLine(
                "line_0", 
                f"大家好，我是{name}！今天想跟你们聊聊{topic}，这个话题啊，其实我想了很久要不要说，因为真的挺有意思的。你们肯定想不到会发生什么，对吧？我跟你们说，这个故事啊，就是那种一开始你觉得很平常，但是到后来你会发现，诶，怎么会这样呢？就特别出乎意料的那种。让我慢慢讲给你们听，你们可得听好了啊。",
                "Hook", 0.3, [topic, "开场", "悬念"]
            ),
            
            # Build-up (3-5单元)
            ScriptLine(
                "line_1",
                f"故事是这样的，那个时候我还在经历一些事情，你们知道吗，就是那种人生的关键时刻。我记得特别特别清楚，当时的情况是这样的。周围的环境啊，人啊，氛围啊，都让我印象深刻。我现在闭上眼睛都能想起来那个画面。有一个很重要的细节，我必须得告诉你们，不然你们可能理解不了后面为什么会那样。就是说，当时我的状态是...",
                "Build-up", 0.5, ["背景铺垫", "场景描述"]
            ),
            
            ScriptLine(
                "line_2",
                "然后呢，事情开始变得有点不一样了。我当时心里就在想，哎呀，这事儿可能要有点不太妙了。你们能理解那种感觉吗？就是你明明知道可能会出问题，但是你又不知道该怎么办，就很纠结，对吧？我那个时候真的是，怎么说呢，心里七上八下的。每天都在想这个事儿，睡觉都睡不好。",
                "Build-up", 0.6, ["内心挣扎", "情绪铺垫"]
            ),
            
            ScriptLine(
                "line_3",
                "结果果然啊，接下来发生了一件事情，就让整个局面完全不一样了。我跟你们讲，那个时刻真的是，我现在想起来都觉得很戏剧化。就是那种你在电视剧里才会看到的情节，居然在我身上发生了。我当时的第一反应就是，天哪，这是真的吗？我是不是在做梦？",
                "Build-up", 0.7, ["转折前兆", "情绪升级"]
            ),
            
            # Climax (2-3单元)
            ScriptLine(
                "line_4",
                "当时我整个人都傻了，你们猜发生了什么？就在那一瞬间，关键的转折来了。我真的是完全没想到会是这样的结果。我的心脏都快跳出来了，手心全是汗，脑子里一片空白。周围的人都在看着我，我都不知道该说什么好。那个场面啊，我告诉你们，真的是尴尬到不行，紧张到不行。",
                "Climax", 0.9, ["高潮", "情绪爆发"]
            ),
            
            ScriptLine(
                "line_5",
                "我告诉你们，结果完全出乎意料。没想到最后竟然是这样，真的是太神奇了。我当时都不知道该笑还是该哭，就是那种复杂的心情，你们懂吗？反正就是，人生啊，真的是充满了意外。你永远不知道下一秒会发生什么。哈哈哈，现在想起来还觉得挺好玩的。",
                "Climax", 0.8, ["结果揭晓", "情绪释放"]
            ),
            
            # Resolution (1-2单元)
            ScriptLine(
                "line_6",
                f"现在想起来还是很感慨，那次关于{topic}的经历真的很特别，给了我很多启发。后来我就明白了一个道理，就是说，有些事情吧，你不经历过是真的不会懂的。所以我就想着分享给你们，希望你们也能从中学到点什么。你们有类似的经历吗？可以在弹幕里告诉我，我很想知道你们的故事。好了，今天就讲到这里，咱们下次再见啦！",
                "Resolution", 0.4, ["总结", "互动", "收尾"]
            ),
        ]


print("✅ 剧本生成器 V2 已定义（预生成完整台词模式）")


### 7.3 ScriptGeneratorV3（主力生成器）⭐ 默认

> ✅ **这是默认生成器，推荐使用**

核心改进：
1. **两阶段沉浸** - 先让 AI 输出"当前状态"（什么触发了回忆、什么心情）
2. **随机 Few-shot** - 从 30 个真实片段中随机采样，确保多样性
3. **情绪断点机制** - 三级情绪断点（微破防/明显破防/完全破防）
4. **"不完美"强化** - 每 200 字必须有 2-3 处认知特征
5. **跑题深度** - 至少 1 段 50 字以上的跑题，然后拉回来
6. **自然卡壳** - 词语重复要断断续续，不是设计感重复

In [79]:
# ==================== ScriptGeneratorV3: 改进版剧本生成器 ====================

class ScriptGeneratorV3:
    """
    剧本生成器 V3 - 两阶段沉浸 + 情绪断点 + 随机 Few-shot
    
    改进点：
    1. Phase 0: 沉浸状态（50-100字描述"当前状态"）
    2. Phase 1: 基于沉浸状态 + 随机 few-shot 生成剧本
    3. 情绪断点: 三级情绪断点规则
    4. "不完美"强化: 每200字必须有2-3处认知特征
    """
    
    # ==================== Phase 0: 沉浸 Prompt ====================
    IMMERSION_PROMPT = """你是 {name}，{persona}。

你正在直播。刚刚发生了一件小事（可能是吃到什么、看到弹幕、或者脑子里突然闪过一个画面），让你想起了"{topic}"。

用50-100字描述你现在的状态：
- 什么触发了这个回忆？具体是什么东西/画面/词语？
- 你现在什么心情？想分享？有点不好意思？觉得好笑？
- 你打算怎么开口？直接说还是铺垫一下？

不要写剧本，只是让我知道你"进入状态"了。用第一人称。"""

    # ==================== System Prompt V3 ====================
    SYSTEM_PROMPT_V3 = '''你是一个专门模拟直播主播**即兴口语**的生成器。

## ⚠️ 核心认知：你不是在写剧本

你是在模拟一个人**边想边说**的过程。想象你现在就坐在直播间，刚刚被一个念头勾起了回忆。你没有提纲，没有彩排，就是现场想现场说。

---

## 🧠 真人说话的认知特征

### 必须包含的"不完美"（每200字至少出现2-3次）

| 类型 | 示例 | 说明 |
|------|------|------|
| 数字模糊 | "三瓶还是四瓶来着"、"大概半小时吧不对可能二十分钟" | 人不会记得精确数字 |
| 自我修正 | "就是我室友，跟我一起同居的，那不叫同居，合租的室友" | 说着说着发现词不对 |
| 词语重复 | "放了那，放了那"、"他就，他就跟我说" | 思考时的填充 |
| 思路中断 | "诶我说到哪了"、"对了这个等会再说" | 被自己带跑 |
| 跑题细节 | "那时候汇率多少来着...好像6点几？" | 联想到无关的事 |

### 🚫 绝对禁止的"AI腔"

❌ "你们能理解那种感觉吗？就是那种..."（解释性元叙述）
❌ "就是那种明知道不对但是还是忍不住的感觉"（解释情绪，不要解释，直接讲事情）
❌ "我觉得我今天必须跟你们分享..."（预谋感开场）
❌ "所以说，人与人之间的善意..."（刻意升华）
❌ 每句话都在推进故事（太紧凑，缺废话）
❌ 精确的时间金额（"500日元"→"几百日元吧"）
❌ 设计感太强的重复（"诱惑我，诱惑我"太工整）

---

## 🌀 跑题深度要求（重要！）

**至少1段要"严重跑题"50字以上**，然后用"诶我说到哪了"拉回来。

### 跑题示例（真实主播）
```
"那个时候汇率很高，那个时候汇率是6.8到7，我去的时候好像是7，好像是我去了半年就降到6.8了，还以为降得很低了呢，谁想到现在的汇率。天呐，爸爸没想到，现在能降这么低。我那个时候等于是...诶我说到哪了？对，腰果的事。"
```

### 可以跑题的方向
- 当时住的公寓长什么样、几楼、窗户朝向
- 超市/药妆店打折时间是几点
- 便宜拉面是什么牌子、多少钱
- 当时用的手机型号、壁纸是什么
- 室友的某个无关习惯

---

## 🗣️ 真实卡壳 vs 设计感重复

**❌ 设计感重复（太工整）**
- "诱惑我，诱惑我"
- "好想吃，好想吃"

**✅ 真实卡壳（断断续续）**
- "就在那里...就在那里一直...一直看着我"
- "我就...我就...就是忍不住"（三次重复）
- "那个味道真的是...真的是那种...怎么说，就是很香"
- "他就，他就跟我说"（自然的思考填充）
- "放了那，放了那"（原地踏步找词）

---

## 🎭 情绪断点机制（Emotion Break）

不是每段都要破防，但在关键时刻**可能**出现：

### 触发条件（随机触发，不是必须）
1. **回忆到具体画面时** - "我现在想起他当时的表情，我就..."
2. **说到关键转折时** - "然后他跟我说...（停顿）...他说..."
3. **意识到自己当时有多傻时** - "我那时候居然...天哪我怎么会..."
4. **感激/愧疚涌上来时** - "他人真的太好了，我..."

### 情绪断点的表现形式

**Level 1: 微微破防（概率40%）**
- 语速突然变快或变慢
- 插入"救命"、"天哪"、"不行我说不下去了"
- 示例："我现在说着说着又想哭了，不是，我继续，我继续说"

**Level 2: 明显破防（概率20%）**
- ⚠️ **文字必须体现说不下去！不是只标注！**
- 句子说不完整，有"..."表示停顿
- 需要插入"不行我说不下去了"或深呼吸
- ❌ 错误："然后我做了一个更加愚蠢的决定"（太流畅）
- ✅ 正确："然后我就...不行我说到这里就想笑自己...我当时居然想说，要不然把整包都吃完"
- ✅ 正确："然后他就...他就跟我说...（深呼吸）他说你想吃就吃呀，我当时就...就..."

**Level 3: 完全破防（概率5%，高潮段落）**
- 语无伦次
- 需要转移话题缓解
- 示例："我不行了我说不下去了，你们等我一下...弹幕别笑我啊，我真的每次说到这里都会..."

### ⚠️ 重要：不要滥用！
- 一个4分钟的剧本，最多1-2处明显破防
- Level 3 只在真正的情感高潮才用
- 没有破防也完全OK，很多故事就是轻松讲完的

---

## 📖 叙事结构

**开场（Hook）- 触发式，不是宣告式**
❌ "我今天必须跟你们分享一个事情"
✅ "我刚才吃那个什么来着...吃了个坚果，突然想起来"

**铺垫（Build-up）- 废话要多**
每段至少30%是"与主线无关但增加真实感"的内容：天气、当时在干嘛、某个无关紧要的细节

**高潮（Climax）- 可以有情绪断点**
心理滑坡的展示：侥幸→试探→失控→慌张

**收尾（Resolution）- 不要升华**
❌ "人与人之间的善意比我们想象的要多"
✅ "后来又给我带了一盒，我吃。对了今天SC有人问什么来着..."

---

{fewshot_examples}

---

## 输出格式

生成8-12个叙事单元，每个200-400字，JSON数组格式：

```json
[
  {{
    "id": "line_0",
    "text": "口语内容（必须包含上述真人特征）",
    "stage": "Hook",
    "cost": 0.3,
    "key_info": ["关键信息1", "关键信息2"],
    "disfluencies": ["数字模糊", "自我修正"],
    "emotion_break": null
  }},
  {{
    "id": "line_5",
    "text": "高潮部分...",
    "stage": "Climax",
    "cost": 0.9,
    "key_info": ["转折点"],
    "disfluencies": ["词语重复", "思路中断"],
    "emotion_break": {{"level": 2, "trigger": "感激涌上来", "recovery": "用自嘲化解"}}
  }}
]
```

### disfluencies 可选值
- "数字模糊": 不确定的数字/时间
- "自我修正": 说错后纠正
- "词语重复": 重复某个词或短语（要自然卡壳，不是设计感重复）
- "思路中断": 跑题或忘词
- "跑题细节": 提到无关但真实的细节
- "严重跑题": 50字以上的跑题，然后拉回来

### ⚠️ 关键规则（必须遵守！）
1. **emotion_break.level >= 2 时，text 必须体现说不下去**（有"..."停顿、有"不行"、句子不完整）
2. **至少1个单元要有"严重跑题"**（50字以上无关细节，然后"诶我说到哪了"拉回来）
3. **词语重复要自然**（"他就...他就跟我说"而不是"诱惑我，诱惑我"）
4. **不要解释情绪**，直接讲事情让观众自己体会
'''

    def __init__(self, llm: LLMClient, example_sampler: ExampleSampler = None):
        """
        初始化 V3 剧本生成器
        
        Args:
            llm: LLM 客户端
            example_sampler: ExampleSampler 实例（可选，用于随机 few-shot）
        """
        self.llm = llm
        self.example_sampler = example_sampler
    
    def generate(self, name: str, persona: str, background: str, 
                 topic: str, language: str = "zh") -> List[ScriptLine]:
        """
        两阶段生成剧本
        
        Args:
            name: 角色名
            persona: 人设描述
            background: 背景故事
            topic: 要讲的话题
            language: 语言 ('zh' 或 'en')
        
        Returns:
            List[ScriptLine]: 生成的剧本台词列表
        """
        
        # === Phase 0: 建立沉浸状态 ===
        print("🎭 Phase 0: 建立沉浸状态...")
        immersion_prompt = self.IMMERSION_PROMPT.format(
            name=name, persona=persona, topic=topic
        )
        immersion_state = self.llm.call(immersion_prompt, max_tokens=200)
        print(f"   沉浸状态:\n   {immersion_state[:150]}...\n")
        
        # === Phase 1: 采样 few-shot examples ===
        fewshot_examples = ""
        if self.example_sampler:
            print("📚 Phase 1: 采样 few-shot examples...")
            sampled_clips = self.example_sampler.sample_diverse(n=3, language=language)
            if sampled_clips:
                fewshot_examples = "## 📚 真实主播说话风格参考\n\n" + self.example_sampler.format_as_fewshot(sampled_clips)
                print(f"   采样了 {len(sampled_clips)} 个 examples")
                for clip in sampled_clips:
                    print(f"   - {clip.get('title', '未知')}")
            else:
                print("   ⚠️ 无可用 examples")
        
        # === Phase 2: 生成剧本 ===
        print("\n✍️ Phase 2: 生成剧本...")
        system = self.SYSTEM_PROMPT_V3.replace("{fewshot_examples}", fewshot_examples)
        
        user_prompt = f"""## 你的当前状态（沉浸感）
{immersion_state}

## 角色信息
- 名字: {name}
- 人设: {persona}
- 背景: {background}
- 话题: {topic}

## 生成要求
- 目标时长: 240秒（4分钟）
- 需要: 8-12个叙事单元
- 每个单元: 200-400字
- 必须体现"边想边说"的认知特征
- 可以有1-2处轻微情绪波动，不强求
- 每个单元必须标注 disfluencies（认知特征）
- 适当位置可以添加 emotion_break（情绪断点）

## ⚠️ 质量检查（严格遵守！）
每个单元必须满足：
1. 至少包含2种 disfluencies
2. 有废话/跑题细节（不是纯叙事推进）
3. 口语化，有停顿，有情绪

## 🔴 必须做到
1. **严重跑题**: 至少1段要跑题50字以上（比如详细说汇率、公寓、打折时间），然后"诶我说到哪了"拉回来
2. **Level 2 破防**: 如果标注了 emotion_break.level >= 2，text 里必须有"..."停顿或"不行我说不下去了"，不能写流畅的句子
3. **自然卡壳**: 词语重复要像"他就...他就跟我说"，不要像"诱惑我，诱惑我"那么工整
4. **不解释情绪**: 不要写"就是那种明知道不对但忍不住的感觉"，直接讲事情

现在基于你的沉浸状态，开始"边想边说"。只输出JSON数组。"""

        response = self.llm.call(user_prompt, system=system, max_tokens=8000)
        
        # === 解析响应 ===
        return self._parse_response(response)
    
    def _parse_response(self, response: str) -> List[ScriptLine]:
        """解析 LLM 响应为 ScriptLine 列表"""
        import re
        
        # 提取 JSON 部分
        json_match = re.search(r'\[[\s\S]*\]', response)
        if not json_match:
            print(f"⚠️ 无法解析 JSON，原始响应:\n{response[:500]}...")
            return []
        
        try:
            data = json.loads(json_match.group())
            lines = []
            
            for i, item in enumerate(data):
                # 兼容 cost 和 interruption_cost 两种命名
                cost = item.get('cost', item.get('interruption_cost', 0.5))
                
                line = ScriptLine(
                    id=item.get('id', f'line_{i}'),
                    text=item.get('text', ''),
                    stage=item.get('stage', 'Build-up'),
                    interruption_cost=cost,
                    key_info=item.get('key_info', []),
                    disfluencies=item.get('disfluencies', []),
                    emotion_break=item.get('emotion_break', None)
                )
                lines.append(line)
            
            print(f"✅ 生成了 {len(lines)} 句台词")
            
            # 质量统计
            total_disfluencies = sum(len(l.disfluencies) for l in lines)
            emotion_breaks = sum(1 for l in lines if l.emotion_break)
            print(f"   认知特征总数: {total_disfluencies}")
            print(f"   情绪断点: {emotion_breaks} 处")
            
            return lines
            
        except json.JSONDecodeError as e:
            print(f"⚠️ JSON 解析错误: {e}")
            print(f"   原始内容: {json_match.group()[:300]}...")
            return []


# ==================== 初始化 ScriptGeneratorV3 ====================
try:
    script_gen_v3 = ScriptGeneratorV3(llm, example_sampler)
    print("✅ ScriptGeneratorV3 已初始化（带随机 few-shot + 情绪断点）")
except Exception as e:
    script_gen_v3 = None
    print(f"⚠️ ScriptGeneratorV3 初始化失败: {e}")

✅ ScriptGeneratorV3 已初始化（带随机 few-shot + 情绪断点）


## Part 8: 表演引擎（带记忆系统）

- 实时表演 + 弹幕互动
- 记忆系统追踪剧本进度、已提到信息、情绪轨迹
- 支持 V3 的 emotion_break 和 disfluencies

In [80]:
class PerformerV2:
    """
    表演引擎 V2 - 带记忆系统和统一弹幕处理
    """
    
    def __init__(self, llm: LLMClient, tts: TTSClient, danmaku_handler: DanmakuHandler):
        self.llm = llm
        self.tts = tts
        self.danmaku_handler = danmaku_handler
    
    def step(self, state: PerformanceState, new_danmaku: List[Danmaku] = None) -> Dict:
        """
        执行一步表演
        
        流程：
        1. 评估弹幕优先级
        2. 决定是否打断当前台词
        3. 如果打断：生成回应 + 承接回剧本
        4. 如果不打断：说当前台词
        5. 更新记忆
        6. 检查承诺是否该兑现
        """
        
        if new_danmaku:
            state.danmaku_queue.extend(new_danmaku)
            # 记录到记忆
            for d in new_danmaku:
                state.memory.danmaku_memory["received"].append(d.text)
        
        # 检查结束
        if state.current_line_idx >= len(state.script_lines):
            return self._generate_ending(state)
        
        current_line = state.script_lines[state.current_line_idx]
        
        # 处理弹幕
        best_danmaku = None
        handle_result = None
        
        if state.danmaku_queue:
            # 找到最高优先级的弹幕
            for danmaku in state.danmaku_queue:
                result = self.danmaku_handler.handle(danmaku, state)
                if result.get("should_interrupt"):
                    if best_danmaku is None or result.get("priority", 0) > handle_result.get("priority", 0):
                        best_danmaku = danmaku
                        handle_result = result
        
        # 决策
        if best_danmaku and handle_result:
            # 打断！回应弹幕
            output = self._handle_danmaku_response(best_danmaku, handle_result, current_line, state)
            
            # 从队列移除
            state.danmaku_queue = [d for d in state.danmaku_queue if d != best_danmaku]
            
            # 记录到记忆
            state.memory.danmaku_memory["responded"].append(best_danmaku.text)
            if best_danmaku.is_question():
                # 如果是问题，检查是否记录承诺
                if handle_result.get("action") == "tease":
                    answer_loc = handle_result.get("answer_loc", {})
                    if answer_loc.get("found"):
                        state.memory.promises.append({
                            "content": best_danmaku.text,
                            "made_at_step": state.current_step,
                            "fulfilled": False,
                            "answer_at_line": answer_loc.get("line_idx")
                        })
        else:
            # 继续当前台词
            output = {
                "speech": current_line.text,
                "action": "continue",
                "priority": 0.0,
                "cost": current_line.interruption_cost
            }
        
        # 更新剧本进度
        state.current_line_idx += 1
        state.current_step += 1
        
        # 更新记忆：记录已提到的信息
        for info in current_line.key_info:
            if info not in state.memory.story_points["mentioned"]:
                state.memory.story_points["mentioned"].append(info)
        
        # 更新剧本进度记忆
        state.memory.script_progress["current_line"] = state.current_line_idx
        state.memory.script_progress["total_lines"] = len(state.script_lines)
        state.memory.script_progress["current_stage"] = current_line.stage
        
        # 检查承诺是否兑现
        self._check_promises(current_line, state)
        
        # 生成语音（带情绪增强）
        speech = output.get("speech", "")
        audio = None
        if speech and self.tts.enabled:
            # 根据剧本阶段动态调整情绪强度
            emotion_boost = {
                "Hook": 0.3,       # 开场：中等情绪
                "Build-up": 0.2,   # 铺垫：较低情绪
                "Climax": 0.8,     # 高潮：最强情绪！
                "Resolution": 0.4  # 收尾：适中情绪
            }.get(current_line.stage, 0.0)
            
            # V3: 如果有情绪断点，增强情绪
            if current_line.emotion_break:
                level = current_line.emotion_break.get('level', 0)
                emotion_boost += level * 0.15  # Level 1 +0.15, Level 2 +0.30, Level 3 +0.45
            
            # 如果是回应SC或高优先级弹幕，增强情绪
            if output.get("action") in ["tease", "improvise", "jump"]:
                emotion_boost += 0.2
            
            emotion_boost = min(1.0, emotion_boost)  # 限制在1.0以内
            audio = self.tts.synthesize(speech, emotion_boost=emotion_boost)
        
        output["audio"] = audio
        output["line_idx"] = state.current_line_idx - 1
        output["stage"] = current_line.stage
        output["step"] = state.current_step
        
        # V3: 添加认知特征和情绪断点
        output["disfluencies"] = current_line.disfluencies
        output["emotion_break"] = current_line.emotion_break
        
        # V3: 追踪情绪轨迹
        if current_line.emotion_break:
            state.memory.emotion_track.append({
                "step": state.current_step,
                "line_idx": state.current_line_idx - 1,
                "level": current_line.emotion_break.get('level', 0),
                "trigger": current_line.emotion_break.get('trigger', ''),
                "stage": current_line.stage
            })
        
        output["memory_display"] = state.memory.to_display()
        
        return output
    
    def _handle_danmaku_response(self, danmaku: Danmaku, handle_result: Dict,
                                  current_line: ScriptLine, state: PerformanceState) -> Dict:
        """处理弹幕回应"""
        
        echo = handle_result.get("echo", "")
        action = handle_result.get("action", "continue")
        answer_loc = handle_result.get("answer_loc", {})
        
        # 生成回应内容
        response_part = ""
        
        if action == "improvise":
            # 即兴回答（剧本里没有，但要积极响应，尤其是SC）
            if danmaku.is_sc:
                # SC必须热情响应，允许跑题
                sc_responses = [
                    f"哇谢谢{danmaku.user}的SC！{danmaku.text}是吧，",
                    f"诶有SC！谢谢{danmaku.user}！关于{danmaku.text}啊，",
                    f"感谢SC！{danmaku.text}...让我想想，",
                ]
                response_part = random.choice(sc_responses) + "这个话题也挺有意思的！虽然跟今天讲的不太一样，但我觉得有点像，就是..."
            else:
                # 普通弹幕也要积极回应
                response_part = f"哈哈{danmaku.text}！说得对，这让我想起来..."
        elif action == "jump":
            # 跳跃到答案（高优先级）
            response_part = f"既然你问了，我直接告诉你！"
            # TODO: 可以考虑跳跃到答案行
        elif action == "tease":
            # 吊胃口
            distance = answer_loc.get("distance", 0)
            if distance <= 2:
                response_part = "诶你别急，马上就要说到了~"
            else:
                response_part = "这个问题太好了！先卖个关子，你们继续听就知道了，绝对出乎意料~"
        else:  # continue
            response_part = "说得对！"
        
        # 承接回剧本
        transition = self._generate_transition(current_line)
        
        speech = f"{echo}{response_part} {transition}{current_line.text}"
        
        return {
            "speech": speech,
            "action": action,
            "danmaku": danmaku.text,
            "priority": handle_result.get("priority", 0),
            "cost": handle_result.get("cost", 0),
            "relevance": handle_result.get("relevance", 0)
        }
    
    def _generate_transition(self, next_line: ScriptLine) -> str:
        """生成承接语"""
        transitions = [
            "好，那刚才说到，",
            "回到我们的故事，",
            "继续说，",
            "对了，",
            ""  # 有时候不需要过渡
        ]
        import random
        return random.choice(transitions)
    
    def _check_promises(self, current_line: ScriptLine, state: PerformanceState):
        """检查当前台词是否兑现了承诺"""
        for promise in state.memory.promises:
            if promise.get("fulfilled", False):
                continue
            
            # 检查当前行是不是答案行
            if promise.get("answer_at_line") == state.current_line_idx - 1:
                promise["fulfilled"] = True
                # 可以在台词中加入提醒："诶对了，刚才有人问XXX，答案来了！"
    
    def _generate_ending(self, state: PerformanceState) -> Dict:
        """生成结尾"""
        speech = f"好啦，今天关于{state.topic}就聊到这里，谢谢大家！"
        audio = self.tts.synthesize(speech) if self.tts.enabled else None
        
        return {
            "speech": speech,
            "action": "end",
            "step": state.current_step,
            "audio": audio,
            "disfluencies": [],  # V3: 结尾无认知特征
            "emotion_break": None,  # V3: 结尾无情绪断点
            "memory_display": state.memory.to_display()
        }


print("✅ 表演引擎 V2 已定义（带记忆系统和统一弹幕处理）")


✅ 表演引擎 V2 已定义（带记忆系统和统一弹幕处理）


## Part 9: 完整引擎

整合所有组件的 EchuuEngineV2：
- Phase 1: 预生成完整剧本（默认使用 V3 生成器）
- Phase 2: 实时表演 + 记忆系统
- 自动保存剧本和语音

In [81]:
class EchuuEngineV2:
    """
    echuu 引擎 - 完整流程
    
    Phase 1: 预生成完整剧本（默认使用 V3 生成器）
    Phase 2: 实时表演 + 记忆系统
    
    剧本生成器选项:
    - use_v3_generator=True（默认）: ScriptGeneratorV3，两阶段沉浸 + 随机 few-shot + 情绪断点
    - use_v3_generator=False: ScriptGeneratorV2，原有生成器
    """
    
    def __init__(self, llm: LLMClient, tts: TTSClient, analyzer: PatternAnalyzer,
                 use_v3_generator: bool = True):
        self.llm = llm
        self.tts = tts
        self.analyzer = analyzer
        self.use_v3 = use_v3_generator
        
        # 选择剧本生成器
        if use_v3_generator:
            # V3: 两阶段沉浸 + 随机 few-shot + 情绪断点
            global example_sampler, script_gen_v3
            if example_sampler is not None:
                self.script_gen = ScriptGeneratorV3(llm, example_sampler)
                print("🎭 使用 ScriptGeneratorV3（带两阶段沉浸 + 情绪断点）")
            else:
                self.script_gen = ScriptGeneratorV3(llm, None)
                print("🎭 使用 ScriptGeneratorV3（无 few-shot，ExampleSampler 未初始化）")
        else:
            # V2: 原有生成器
            self.script_gen = ScriptGeneratorV2(llm, analyzer)
            print("📝 使用 ScriptGeneratorV2")
        
        self.performer = PerformerV2(llm, tts, danmaku_handler)
        
        # 创建剧本保存目录
        self.scripts_dir = PROJECT_ROOT / "output" / "scripts"
        self.scripts_dir.mkdir(parents=True, exist_ok=True)
    
    def _save_script(self, script_lines, name: str, topic: str):
        """保存剧本到 JSON 文件"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        safe_topic = topic[:30].replace(' ', '_').replace('/', '_')
        filename = f"{timestamp}_{name}_{safe_topic}.json"
        filepath = self.scripts_dir / filename
        
        script_data = {
            "metadata": {
                "timestamp": timestamp,
                "name": name,
                "topic": topic,
                "total_lines": len(script_lines),
            },
            "script": [
                {
                    "id": line.id,
                    "text": line.text,
                    "stage": line.stage,
                    "cost": line.interruption_cost,
                    "key_info": line.key_info,
                    "disfluencies": line.disfluencies,  # V3: 认知特征
                    "emotion_break": line.emotion_break  # V3: 情绪断点
                }
                for line in script_lines
            ]
        }
        
        with open(filepath, "w", encoding="utf-8") as f:
            json.dump(script_data, f, ensure_ascii=False, indent=2)
        print(f"💾 剧本已保存: {filepath}")
    
    def create_performance(self, name: str, persona: str, background: str,
                          topic: str, language: str = "zh") -> PerformanceState:
        """
        Phase 1: 创建表演（预生成完整剧本）
        """
        print(f"\n{'='*60}")
        print(f"🎬 echuu v2 - 预生成完整剧本")
        print(f"{'='*60}\n")
        
        print(f"角色: {name}")
        print(f"话题: {topic}\n")
        
        print("⏳ 正在生成100秒完整剧本...")
        script_lines = self.script_gen.generate(name, persona, background, topic, language)
        
        # 💾 保存剧本到文件
        self._save_script(script_lines, name, topic)
        
        # 显示剧本
        print(f"\n📖 生成的剧本：")
        print(f"{'='*60}")
        for i, line in enumerate(script_lines):
            cost_bar = "█" * int(line.interruption_cost * 5) + "░" * (5 - int(line.interruption_cost * 5))
            print(f"\n[{i}] {line.stage} {cost_bar} cost={line.interruption_cost:.1f}")
            print(f"    {line.text[:80]}{'...' if len(line.text) > 80 else ''}")
            print(f"    🔑 key_info: {', '.join(line.key_info)}")
        print(f"\n{'='*60}\n")
        
        # 提取口癖
        catchphrases = [cp for cp, _ in self.analyzer.extract_catchphrases(language)[:5]]
        
        # 初始化记忆
        memory = PerformerMemory()
        memory.script_progress["total_lines"] = len(script_lines)
        memory.script_progress["current_stage"] = script_lines[0].stage if script_lines else "Unknown"
        
        # 提取即将要说的关键点
        for line in script_lines:
            memory.story_points["upcoming"].extend(line.key_info)
        
        return PerformanceState(
            name=name,
            persona=persona,
            background=background,
            topic=topic,
            script_lines=script_lines,
            memory=memory,
            catchphrases=catchphrases
        )
    
    def run(self, state: PerformanceState, danmaku_sim: List[Dict] = None,
            play_audio: bool = True, save_audio: bool = True) -> List[Dict]:
        """
        Phase 2: 运行表演（实时模式）
        
        Args:
            state: 表演状态
            danmaku_sim: 模拟弹幕
            play_audio: 是否逐句播放音频
            save_audio: 是否保存完整直播音频
        """
        
        results = []
        danmaku_by_step = defaultdict(list)
        
        if danmaku_sim:
            for d in danmaku_sim:
                step = d.get("step", 0)
                danmaku_by_step[step].append(Danmaku.from_text(d.get("text", "")))
        
        # 🔴 开始录制
        if save_audio and self.tts.enabled:
            self.tts.start_recording()
        
        print(f"\n{'='*60}")
        print(f"🎭 开始实时表演")
        if save_audio and self.tts.enabled:
            print(f"🔴 正在录制...")
        print(f"{'='*60}\n")
        
        for step in range(len(state.script_lines)):
            # 获取新弹幕
            new_danmaku = danmaku_by_step.get(step, [])
            
            # 执行一步
            result = self.performer.step(state, new_danmaku)
            results.append(result)
            
            # 显示
            step_num = result.get("step", 0)
            stage = result.get("stage", "?")
            action = result.get("action", "continue")
            speech = result.get("speech", "")
            
            action_icons = {
                "continue": "📖",
                "tease": "🎣",
                "jump": "⚡",
                "improvise": "🎲",
                "end": "🎭"
            }
            icon = action_icons.get(action, "📖")
            
            print(f"[Step {step_num}] {stage} {icon} {action.upper()}")
            print(f"  📢 {speech[:100]}{'...' if len(speech) > 100 else ''}")
            
            if result.get("danmaku"):
                print(f"  💬 回应弹幕: {result['danmaku']}")
                print(f"  📊 priority={result.get('priority', 0):.2f}, cost={result.get('cost', 0):.2f}, relevance={result.get('relevance', 0):.2f}")
            
            # V3: 显示认知特征和情绪断点
            if result.get('emotion_break'):
                level = result['emotion_break'].get('level', 0)
                level_name = {1: "微破防", 2: "明显破防", 3: "完全破防"}.get(level, f"L{level}")
                trigger = result['emotion_break'].get('trigger', '')
                print(f"  🎭 情绪断点: {level_name} - {trigger}")
            if result.get('disfluencies'):
                print(f"  💬 认知特征: {', '.join(result['disfluencies'])}")
            
            # 显示记忆（每3步显示一次）
            if step_num % 3 == 0:
                print(f"\n{result.get('memory_display', '')}")
            
            # 🔊 逐句播放音频（关键！）
            if play_audio and result.get('audio'):
                print(f"  🔊 播放语音...")
                display(Audio(result['audio'], autoplay=True))
                # 可以添加延迟，等待播放完成
                # import time
                # time.sleep(len(result['audio']) / 10000)  # 估算播放时长
            
            print()
            
            if action == "end":
                break
        
        print(f"\n{'='*60}")
        print(f"✅ 表演结束！")
        print(f"{'='*60}\n")
        
        # 💾 保存录制的音频
        if save_audio and self.tts.enabled:
            from datetime import datetime
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            audio_path = self.scripts_dir / f"{timestamp}_{state.name}_{state.topic[:20].replace(' ', '_')}_live.mp3"
            self.tts.save_recording(str(audio_path))
        
        # 最终记忆状态
        print("🧠 最终记忆状态：")
        print(state.memory.to_display())
        
        return results


# 初始化引擎（默认使用 V3 生成器）
engine_v2 = EchuuEngineV2(llm, tts, analyzer)  # use_v3_generator=True 是默认值
print("\n✅ echuu 引擎初始化完成（默认使用 ScriptGeneratorV3）！")


🎭 使用 ScriptGeneratorV3（带两阶段沉浸 + 情绪断点）

✅ echuu 引擎初始化完成（默认使用 ScriptGeneratorV3）！


## Part 10: 🚀 快速开始

运行测试用例，体验完整流程。

In [82]:
# 测试用例：经典的室友腰果故事

test_case_v2 = {
    "name": "六螺",
    "persona": "25岁主播，活泼自嘲，喜欢分享生活经历，口癖：我觉得、对吧、就是说",
    "background": "做过很多工作，现在是全职主播，留学日本多年",
    "topic": "留学时偷吃室友腰果的故事",
    "danmaku": [
        {"step": 1, "text": "哈哈哈"},
        {"step": 2, "text": "我也有类似经历"},
        {"step": 4, "text": "室友知道吗？"},  # 高相关问题
        {"step": 6, "text": "[SC ¥50] 后来室友生气了吗"},  # SC + 高相关
        {"step": 8, "text": "笑死"},
    ]
}

# Phase 1: 预生成完整剧本
state_v2 = engine_v2.create_performance(
    name=test_case_v2["name"],
    persona=test_case_v2["persona"],
    background=test_case_v2["background"],
    topic=test_case_v2["topic"],
    language="zh"
)



🎬 echuu v2 - 预生成完整剧本

角色: 六螺
话题: 留学时偷吃室友腰果的故事

⏳ 正在生成100秒完整剧本...
🎭 Phase 0: 建立沉浸状态...
   沉浸状态:
   刚刚咬了一口坚果酸奶，那个腰果的味道瞬间就把我拉回到留学那会儿了！我觉得我现在脸都有点发烫，但是又忍不住想笑，就是说那段黑历史实在是太丢人了，对吧！不过直播间气氛这么好，我觉得可以跟大家分享一下，反正都是老朋友了。我打算先卖个关子，问问大家有没有做过什么偷偷摸摸又心虚的事情，然后再坦白我那个超级社死...

📚 Phase 1: 采样 few-shot examples...
   采样了 3 个 examples
   - 沉没成本与读博换导师
   - 转行当老师的畅想
   - 辅威联动（防诈骗现场演示）

✍️ Phase 2: 生成剧本...
✅ 生成了 10 句台词
   认知特征总数: 22
   情绪断点: 2 处
💾 剧本已保存: d:\vtuberclip\echuu-agent\output\scripts\20260125_233213_六螺_留学时偷吃室友腰果的故事.json

📖 生成的剧本：

[0] Hook █░░░░ cost=0.3
    我刚才吃那个什么来着...吃了个坚果酸奶，那个腰果的味道，我的妈呀，瞬间就把我拉回到留学那会儿了。你们有没有做过什么偷偷摸摸又心虚的事情啊？就是说，明知道不太好...
    🔑 key_info: 腰果引发回忆, 询问观众经历

[1] Build-up ██░░░ cost=0.4
    那个时候我在日本嘛，跟一个室友合租，她是个特别好的人，就是说那种好到让你觉得自己很罪恶的程度。她经常从老家带一些零食过来，什么坚果啊、那个什么干果啊，都放在厨房...
    🔑 key_info: 日本合租, 室友很好, 零食在厨房

[2] Build-up ██░░░ cost=0.5
    然后有一天，她出门了，说是去打工还是去上课来着，反正就是不在家。我当时在写那个什么作业，写到一半突然就饿了，就是说那种突然想吃点什么的感觉，你们懂吧？我就想起来...
    🔑 key_info: 室友出门, 自己饿了, 想吃腰果

[3] Build-up ███░░ cost=0.6
    然后我就..

In [ ]:
# Phase 2: 运行实时表演（带弹幕互动）

results_v2 = engine_v2.run(
    state_v2, 
    danmaku_sim=test_case_v2["danmaku"],
    play_audio=True  # 设为 False 可以禁用自动播放
)


🔴 开始录制直播音频...

🎭 开始实时表演
🔴 正在录制...

[TTS] 错误: websocket connection could not established within 5s. Please check your network connection, firewall settings, or server status.
[Step 1] Hook 📖 CONTINUE
  📢 我刚才吃那个什么来着...吃了个坚果酸奶，那个腰果的味道，我的妈呀，瞬间就把我拉回到留学那会儿了。你们有没有做过什么偷偷摸摸又心虚的事情啊？就是说，明知道不太好，但是又控制不住自己的那种？弹幕打个1让...
  💬 认知特征: 思路中断, 自我修正



Traceback (most recent call last):
  File "C:\Users\95801\AppData\Local\Temp\ipykernel_29272\3486252516.py", line 182, in synthesize
    qwen_tts.connect()
  File "d:\vtuberclip\echuu-agent\.venv\lib\site-packages\dashscope\audio\qwen_tts_realtime\qwen_tts_realtime.py", line 156, in connect
    raise TimeoutError(
TimeoutError: websocket connection could not established within 5s. Please check your network connection, firewall settings, or server status.


[Step 2] Build-up 📖 CONTINUE
  📢 那个时候我在日本嘛，跟一个室友合租，她是个特别好的人，就是说那种好到让你觉得自己很罪恶的程度。她经常从老家带一些零食过来，什么坚果啊、那个什么干果啊，都放在厨房的那个...那个柜子里面。我们那个公寓是...
  💬 认知特征: 数字模糊, 词语重复, 跑题细节
  🔊 播放语音...



[Step 3] Build-up 📖 CONTINUE
  📢 然后有一天，她出门了，说是去打工还是去上课来着，反正就是不在家。我当时在写那个什么作业，写到一半突然就饿了，就是说那种突然想吃点什么的感觉，你们懂吧？我就想起来她柜子里有腰果。我就想，就吃一颗，就一颗...
  💬 认知特征: 数字模糊, 自我修正, 词语重复

┌─────────────────────────────────────────────┐
│ 🧠 AI记忆状态                               │
├─────────────────────────────────────────────┤
│ 📖 剧本: [███░░░░░░░] 3/10 (Build-up)  │
│ 💬 弹幕: 已回应0条, 待回答0个问题         │
│ ✅ 已提到:                                  │
│    ✓ 室友出门...                      │
│    ✓ 自己饿了...                      │
│    ✓ 想吃腰果...                      │
└─────────────────────────────────────────────┘
  🔊 播放语音...



[Step 4] Build-up 📖 CONTINUE
  📢 然后我就...我就悄悄地走到厨房，打开那个柜子。那个腰果是装在一个透明的罐子里的，看起来就特别香，对吧？我告诉自己就吃一颗，结果...结果你们猜怎么着？我吃了一颗之后，就觉得，诶，再吃一颗应该也看不出...
  💬 认知特征: 词语重复, 自我修正
  🔊 播放语音...


...


[Step 8] Resolution 🎲 IMPROVISE
  📢 诶有SC！有人说：[SC ¥50] 后来室友生气了吗诶有SC！谢谢观众！关于[SC ¥50] 后来室友生气了吗啊，这个话题也挺有意思的！虽然跟今天讲的不太一样，但我觉得有点像，就是... 继续说，她敲...
  💬 回应弹幕: [SC ¥50] 后来室友生气了吗
  📊 priority=0.90, cost=0.70, relevance=0.00
  🎭 情绪断点: 明显破防 - 被室友的善良感动
  💬 认知特征: 词语重复, 自我修正
  🔊 播放语音...



[Step 9] Resolution 📖 CONTINUE
  📢 然后我就跟她坦白了，说其实我吃了很多，可能吃了一半，我明天去买新的赔给你。她就说不用不用，她本来就是想跟我分享的。然后她还说，下次你想吃什么都可以直接跟我说，不用这么偷偷摸摸的。我当时真的快哭了，就是...
  💬 认知特征: 词语重复, 数字模糊

┌─────────────────────────────────────────────┐
│ 🧠 AI记忆状态                               │
├─────────────────────────────────────────────┤
│ 📖 剧本: [█████████░] 9/10 (Resolution)  │
│ 💬 弹幕: 已回应1条, 待回答0个问题         │
│ 🎭 情绪轨迹:                                │
│    微破防: 回忆起当时的心虚...              │
│    明显破防: 被室友的善良感动...              │
│ ✅ 已提到:                                  │
│    ✓ 坦白了...                      │
│    ✓ 室友更加善良...                      │
│    ✓ 温暖回忆...                      │
└─────────────────────────────────────────────┘
  🔊 播放语音...



[Step 10] Resolution 📖 CONTINUE
  📢 所以说，有时候我们以为的偷偷摸摸，其实对方早就知道了，只是选择用善意来回应。我现在每次吃腰果都会想起这件事，就是说那种...那种被人无条件信任的感觉真的很珍贵，对吧？好了，这就是我超级社死的腰果事件。...
  💬 认知特征: 词语重复, 思路中断
  🔊 播放语音...




✅ 表演结束！

💾 直播音频已保存到: d:\vtuberclip\echuu-agent\output\scripts\20260125_233515_六螺_留学时偷吃室友腰果的故事_live.mp3
   总时长: 340.5秒
🧠 最终记忆状态：
┌─────────────────────────────────────────────┐
│ 🧠 AI记忆状态                               │
├─────────────────────────────────────────────┤
│ 📖 剧本: [██████████] 10/10 (Resolution)  │
│ 💬 弹幕: 已回应1条, 待回答0个问题         │
│ 🎭 情绪轨迹:                                │
│    微破防: 回忆起当时的心虚...              │
│    明显破防: 被室友的善良感动...              │
│ ✅ 已提到:                                  │
│    ✓ 总结感悟...                      │
│    ✓ 每次吃腰果都想起...                      │
│    ✓ 结束话题...                      │
└─────────────────────────────────────────────┘


websocket closed due to fin=1 opcode=8 data=b'\x03\xe8Bye'
websocket closed due to fin=1 opcode=8 data=b'\x03\xe8Bye'


## 附录 A: 🎙️ TTS 高级功能

详细的 TTS 功能演示和使用指南。


In [ ]:
# 🎙️ TTS 增强功能演示

print("=" * 60)
print("🎙️ TTS 增强功能演示")
print("=" * 60)

# 1️⃣ 音色列表
print(f"\n📋 可用音色 ({len(tts.AVAILABLE_VOICES)}个):")
print("女声:", [v for v in tts.AVAILABLE_VOICES if "xiao" in v or v in ["longwan", "longyue", "longjing", "longanyang"]][:5])
print("男声:", [v for v in tts.AVAILABLE_VOICES if v in ["longye", "longfei", "longxiaohao", "longxiaoming"]])
print("特色:", [v for v in tts.AVAILABLE_VOICES if "miaomiao" in v or "lele" in v or "tiantian" in v or "dada" in v])

# 2️⃣ 快速测试音色
print("\n🔊 快速测试音色:")
print("使用方法: test_voice('音色名', '测试文本')")
print("示例:")
print("  test_voice('longmiaomiao', '喵~ 大家好呀，我是小猫猫主播！')")
print("  test_voice('longye', '各位观众朋友们好，今天来给大家讲个故事')")
print("  test_voice('longtiantian', '嗨~ 宝贝们，今天的直播超级精彩哦！')")

# 3️⃣ 情绪参数控制
print("\n🎛️ 情绪参数控制:")
print("tts.set_emotion_params(rate=1.2, pitch=100, volume=80)  # 快语速、高音调、大音量")
print("tts.set_emotion_params(rate=0.8, pitch=-50, volume=50)  # 慢语速、低音调、正常音量")

# 4️⃣ 情绪增强参数
print("\n🔥 情绪增强 (emotion_boost):")
print("emotion_boost=0.0  →  正常语速")
print("emotion_boost=0.5  →  轻微情绪波动（±25%语速，±100音调）")
print("emotion_boost=1.0  →  强烈情绪波动（±50%语速，±200音调）")
print("\n示例:")
print("  tts.synthesize('真的太好笑了！', emotion_boost=0.8)")

# 5️⃣ 直播录制功能
print("\n💾 直播录制功能:")
print("在 engine_v2.run() 中自动开启:")
print("  - 每句语音自动保存到缓存")
print("  - 直播结束自动合并为完整MP3")
print("  - 保存路径: output/scripts/{timestamp}_{name}_{topic}_live.mp3")

print("\n" + "=" * 60)
print("✅ 所有功能已就绪！")
print("=" * 60)


### A.1 📚 TTS 使用指南

### 1️⃣ 音色切换

```python
# 方法1: 使用 test_voice() 快速测试
test_voice('longmiaomiao', '喵~ 这是萌系音色测试')
test_voice('longye', '这是男声测试')

# 方法2: 修改 TTS 实例的音色
tts.change_voice('longtiantian')  # 切换到甜美音色
```

**可用音色分类：**
- **女声**：`longwan`, `longyue`, `longxiaochun`, `longjing`, `longxiaoxin`, `longxiaoya`, `longxiaoyi`, `longanyang` 等
- **男声**：`longye`, `longfei`, `longxiaohao`, `longxiaoming`, `longxiaochen`, `longxiaojun`, `longxiaobo`
- **特色**：`longmiaomiao`(萌系), `longlele`(活泼), `longtiantian`(甜美), `longdada`(大气)

---

### 2️⃣ 情绪参数调整

```python
# 设置基础情绪参数
tts.set_emotion_params(
    rate=1.2,    # 语速 0.5-2.0 (1.0=正常, 1.5=快, 0.8=慢)
    pitch=100,   # 音调 -500到500 (0=正常, +200=高音, -200=低音)
    volume=80    # 音量 0-100
)
```

**情绪增强参数（在synthesize中使用）：**
```python
# emotion_boost: 0.0-1.0
audio = tts.synthesize("天哪！太好笑了！", emotion_boost=0.8)
# 0.0 = 正常
# 0.5 = 中等情绪（±25%语速，±100音调随机波动）
# 1.0 = 最强情绪（±50%语速，±200音调随机波动）
```

**在剧本中自动应用：**
- `Hook` 开场：emotion_boost=0.3
- `Build-up` 铺垫：emotion_boost=0.2
- `Climax` 高潮：emotion_boost=0.8 🔥
- `Resolution` 收尾：emotion_boost=0.4

---

### 3️⃣ 直播录制功能

**自动录制（推荐）：**
```python
# 在 engine_v2.run() 时自动录制
results = engine_v2.run(
    state,
    danmaku_sim=test_danmaku,
    play_audio=True,   # 逐句播放
    save_audio=True    # 自动保存完整录音
)
# 保存路径: output/scripts/{timestamp}_{name}_{topic}_live.mp3
```

**手动录制：**
```python
# 开始录制
tts.start_recording()

# 生成多句语音（自动添加到缓存）
tts.synthesize("第一句话")
tts.synthesize("第二句话")
tts.synthesize("第三句话")

# 保存录制
tts.save_recording("output/my_recording.mp3")
```

---

### 4️⃣ 完整示例

```python
# 1. 切换音色
tts.change_voice('longmiaomiao')  # 使用萌系音色

# 2. 调整情绪参数
tts.set_emotion_params(rate=1.1, pitch=50, volume=70)

# 3. 生成角色剧本
state = engine_v2.create_performance(
    name="小猫猫",
    persona="萌系猫娘主播，说话带喵~",
    background="大学生，喜欢分享日常",
    topic="今天被室友发现偷吃零食的糗事",
    language="zh"
)

# 4. 运行直播（自动录制）
results = engine_v2.run(
    state,
    danmaku_sim=[
        {"step": 2, "text": "好可爱喵~"},
        {"step": 4, "text": "[SC ¥100] 室友没有生气吧？"},
    ],
    play_audio=True,   # 逐句播放音频
    save_audio=True    # 自动保存完整录音
)

# 5. 录音自动保存到: output/scripts/{timestamp}_小猫猫_今天被室友发现偷吃零食的糗_live.mp3
```

---

### 💡 提示

1. **pydub 依赖**：为了正确合并音频，需要安装：
   ```bash
   pip install pydub
   ```

2. **音色测试**：建议先用 `test_voice()` 快速测试不同音色，找到最适合角色的声音

3. **情绪起伏**：`emotion_boost` 会在剧本的 `Climax` 阶段自动提升到 0.8，制造情绪高潮

4. **逐句播放**：每句台词生成后立即播放，模拟真实直播效果


In [ ]:
# 💾 保存剧本（可选）
save_script_to_file(state_v2.script_lines, test_case_v2["name"], test_case_v2["topic"], scripts_dir)

# 📊 查看剧本进度
show_script_progress(state_v2)


## 附录 B: 🎨 自定义角色模板

自定义你的角色和话题进行测试。

In [ ]:
# 🎨 自定义你的角色和话题！

my_test = {
    "name": "小梅",  # 👈 修改名字
    "persona": "温柔的猫娘主播，说话带喵~，喜欢分享温暖的故事",  # 👈 修改人设
    "background": "大学毕业，现在是全职主播",  # 👈 修改背景
    "topic": "第一次养猫的经历",  # 👈 修改话题
    "danmaku": [
        {"step": 1, "text": "好可爱~"},
        {"step": 3, "text": "猫咪叫什么名字？"},  # 高相关问题
        {"step": 5, "text": "[SC ¥100] 猫咪现在多大了"},  # 大额SC
        {"step": 7, "text": "我也想养猫"},
    ]
}

# Phase 1: 生成剧本
my_state = engine_v2.create_performance(
    name=my_test["name"],
    persona=my_test["persona"],
    background=my_test["background"],
    topic=my_test["topic"],
    language="zh"
)


In [ ]:
# Phase 2: 运行表演
my_results = engine_v2.run(
    my_state,
    danmaku_sim=my_test["danmaku"],
    play_audio=True
)


## Part 11: 🧪 V3 质量检测

测试 V3 生成器的改进效果：
1. 检查认知特征（disfluencies）覆盖率
2. 检查情绪断点（emotion_break）分布
3. 验证"不完美"特征是否足够
4. 生成质量报告

In [ ]:
# ==================== V3 效果测试 ====================

def test_v3_quality(script_lines: List[ScriptLine], verbose: bool = True):
    """
    测试 V3 生成的剧本质量
    
    检查：
    1. 认知特征（disfluencies）覆盖率
    2. 情绪断点（emotion_break）分布
    3. 文本中的"不完美"特征
    """
    print("=" * 60)
    print("🧪 V3 剧本质量检查")
    print("=" * 60)
    
    # 统计
    total_lines = len(script_lines)
    total_chars = sum(len(line.text) for line in script_lines)
    
    # 认知特征统计
    disfluency_types = {}
    lines_with_disfluencies = 0
    for line in script_lines:
        if line.disfluencies:
            lines_with_disfluencies += 1
            for d in line.disfluencies:
                disfluency_types[d] = disfluency_types.get(d, 0) + 1
    
    # 情绪断点统计
    emotion_breaks = [line for line in script_lines if line.emotion_break]
    emotion_levels = {}
    for line in emotion_breaks:
        level = line.emotion_break.get('level', 0)
        emotion_levels[level] = emotion_levels.get(level, 0) + 1
    
    # 文本特征检测（真实特征，不是标注的）
    zh_markers = {
        '数字模糊': ['来着', '大概', '好像', '还是', '应该是', '可能'],
        '自我修正': ['不对', '不是', '我的意思是', '那不叫', '我是说'],
        '跑题': ['对了', '说起这个', '诶', '话说', '等等'],
        '情绪词': ['救命', '天哪', '我的天', '哎呀', '卧槽', '太'],
        '词语重复': []  # 需要特殊检测
    }
    
    detected_features = {k: 0 for k in zh_markers}
    lines_with_features = 0
    
    for line in script_lines:
        text = line.text
        has_feature = False
        
        for feature_type, markers in zh_markers.items():
            for marker in markers:
                if marker in text:
                    detected_features[feature_type] += 1
                    has_feature = True
                    break
        
        # 检测词语重复（简单版：检查重复的短语）
        if '，' in text:
            parts = text.split('，')
            for i in range(len(parts)-1):
                if len(parts[i]) >= 2 and len(parts[i+1]) >= 2:
                    if parts[i][-2:] == parts[i+1][:2]:
                        detected_features['词语重复'] += 1
                        has_feature = True
                        break
        
        if has_feature:
            lines_with_features += 1
    
    # 输出结果
    print(f"\n📊 基础统计:")
    print(f"   总行数: {total_lines}")
    print(f"   总字数: {total_chars}")
    print(f"   平均每行字数: {total_chars // total_lines if total_lines else 0}")
    
    print(f"\n📝 认知特征标注 (disfluencies):")
    print(f"   有标注的行: {lines_with_disfluencies}/{total_lines} ({lines_with_disfluencies*100//total_lines if total_lines else 0}%)")
    if disfluency_types:
        for dtype, count in sorted(disfluency_types.items(), key=lambda x: -x[1]):
            print(f"   - {dtype}: {count}")
    else:
        print("   ⚠️ 无认知特征标注")
    
    print(f"\n🎭 情绪断点 (emotion_break):")
    print(f"   总数: {len(emotion_breaks)}")
    if emotion_levels:
        for level, count in sorted(emotion_levels.items()):
            level_name = {1: "微破防", 2: "明显破防", 3: "完全破防"}.get(level, f"Level {level}")
            print(f"   - {level_name}: {count}")
    else:
        print("   （无情绪断点，这可能是正常的）")
    
    print(f"\n🔍 实际文本特征检测:")
    print(f"   有特征的行: {lines_with_features}/{total_lines} ({lines_with_features*100//total_lines if total_lines else 0}%)")
    for feature_type, count in sorted(detected_features.items(), key=lambda x: -x[1]):
        status = "✅" if count > 0 else "⚠️"
        print(f"   {status} {feature_type}: {count}")
    
    # 质量评估
    print(f"\n📈 质量评估:")
    
    # 每200字的认知特征密度
    expected_per_200 = 2.5  # 目标：每200字2-3个
    actual_density = sum(detected_features.values()) / (total_chars / 200) if total_chars else 0
    density_status = "✅" if actual_density >= 1.5 else "⚠️"
    print(f"   {density_status} 认知特征密度: {actual_density:.1f} 个/200字 (目标: ≥2)")
    
    # 情绪断点合理性
    if total_lines >= 8:
        expected_breaks = 1  # 4分钟剧本至少1-2处
        break_status = "✅" if len(emotion_breaks) >= expected_breaks else "💡"
        print(f"   {break_status} 情绪断点: {len(emotion_breaks)} 处 (建议: 1-2处)")
    
    # 详细输出
    if verbose:
        print(f"\n📜 详细内容:")
        print("-" * 60)
        for i, line in enumerate(script_lines):
            text = line.text
            
            # 检查认知特征
            features = []
            if line.disfluencies:
                features.extend(line.disfluencies)
            
            emotion_str = ""
            if line.emotion_break:
                level = line.emotion_break.get('level', 0)
                trigger = line.emotion_break.get('trigger', '')
                emotion_str = f" [情绪L{level}: {trigger}]"
            
            features_str = f" [{', '.join(features)}]" if features else " [⚠️无标注]"
            
            print(f"\n[{line.stage}] Line {i}{emotion_str}")
            print(f"特征:{features_str}")
            print(f"文本: {text[:150]}{'...' if len(text) > 150 else ''}")
    
    return {
        'total_lines': total_lines,
        'total_chars': total_chars,
        'lines_with_disfluencies': lines_with_disfluencies,
        'disfluency_types': disfluency_types,
        'emotion_breaks': len(emotion_breaks),
        'detected_features': detected_features
    }


# ==================== 运行测试 ====================
print("🎬 测试 ScriptGeneratorV3")
print("=" * 60)

# 测试案例
test_case = {
    "name": "六螺",
    "persona": "温柔但偶尔毒舌的女主播，有一只猫叫小橘",
    "background": "曾在日本留学，现在是全职主播",
    "topic": "留学时偷吃室友腰果的故事"
}

# 使用 V3 生成器创建引擎
print("\n1️⃣ 初始化 V3 引擎...")
engine_v3 = EchuuEngineV2(llm, tts, analyzer, use_v3_generator=True)

# 生成剧本
print("\n2️⃣ 生成剧本...")
state = engine_v3.create_performance(**test_case, language="zh")

# 质量检查
print("\n3️⃣ 质量检查...")
if state.script_lines:
    quality_report = test_v3_quality(state.script_lines, verbose=True)
else:
    print("⚠️ 未生成剧本，请检查 LLM 配置")